# Laboratorio LSTM-GRU

En el presente cuaderno construirás dos redes neuronales recurrentes, una LSTM (memoria larga a corto plazo) y una GRU (unidad recurrente cerrada). Aplicando ambos modelos al modelamiento y predicción de la inflación en Colombia. Las LSTM fueron introducidas por [Hochreiter & Schmidhuber](https://direct.mit.edu/neco/article-abstract/9/8/1735/6109/Long-Short-Term-Memory) con el objetivo de mitigar el problema del desvanecimiento de gradientes en redes neuronales recurrentes, mientras que las GRU fueron introducidas por [Cho et al.](https://arxiv.org/pdf/1412.3555) como una versión simplificada de la memoria larga a corto plazo.

**Al finalizar este laboratorio serás capaz de:**
- Implementar estados ocultos de arquitecturas LSTM y GRU utilizando Numpy y Keras.
- Implementar dos modelos completos recurrentes para la predicción de series de tiempo.
- Entender el funcionamiento y el potencial de las redes neuronales recurrentes aplicado a problemas económicos, financieros y actuariales.

Para este laboratorio utilizarás Keras y Numpy.

Antes de ver el problema, corre la celda de abajo para cargar las librerias requeridas y los archivos auxiliares desde el [repositorio](https://github.com/MateoOrtiz001/Laboratory-LSTM-GRU-Inflation).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
!git clone https://github.com/MateoOrtiz001/Laboratory-LSTM-GRU-Inflation
sys.path.append('Laboratory-LSTM-GRU-Inflation')
from utils import *
from tester import *

## Tabla de Contenido
- [1 - Dataset](#1)
- [2 - Problemas de gradientes en redes neuronales recurrentes](#2)
- [3 - Construyendo una LSTM](#3)
  - [3.1 - Construyendo el forward pass de una celda](#3-1)
  - [3.2 - Construyendo el forward pass del modelo](#3-2)
  - [3.3 - Construyendo la retropropagación de una celda](#3-3)
  - [3.4 - Construyendo la repropropagación del modelo](#3-4)
  - [3.5 - Entrenamiento del modelo](#3-5)
  - [3.6 - Predicción del modelo](#3-6)
- [4 - Construyendo una GRU](#4)
  - [4.1 - Construyendo el forward pass de una celda](#4-1)
  - [4.2 - Construyendo el forward pass del modelo](#4-2)
  - [4.3 - Construyendo la retropropagación de una celda](#4-3)
  - [4.4 - Construyendo la repropropagación del modelo](#4-4)
  - [4.5 - Entrenamiento del modelo](#4-5)
  - [4.6 - Predicción del modelo](#4-6)
- [5 - Implementaciones con Keras](#5)
- [6 - Predicciones con intervalos de confianza (Opcional)](#6)
- [7 - Bibliografía](#7)

<a name="1"></a>
## Dataset
El dataset seleccionado corresponde al histórico del indice mensual de la inflación en Colombia presentado por el banco de la república (disponible en su [base de datos](https://suameca.banrep.gov.co/buscador-de-series/#/)).

Primero vamos a cargar el dataset directamente desde el repositorio.

In [ ]:
data = pd.read_excel('Laboratory-LSTM-GRU-Inflation/dataset/Datos-Inflacion.xlsx', decimal=',')

plt.figure(figsize=(10, 6))
plt.plot(data['Inflacion'])
plt.title('Inflacion a traves del tiempo')
plt.xlabel('Fecha')
plt.ylabel('Inflacion')
plt.grid(True)
plt.show()

El conjunto de datos contiene el historial de inflación desde 1955. En primer lugar, recortaremos la información a partir del año 2000, ya que los patrones observados en los registros más antiguos difieren de los de las últimas dos décadas. Posteriormente, prepararemos las columnas para estandarizar los tipos de datos.

In [ ]:
# Convertimos la columna 'Fecha' a objeto datetime y la seleccionamos como índice
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%d/%m/%Y', errors='coerce')
data = data.dropna(subset=['Fecha']).set_index('Fecha')

# Eliminamos filas sin datos en la columna 'Inflacion'
data = data.dropna(subset=['Inflacion']).copy()

# Dividimos los datos de la inflacion sobre 100 para tener el porcentaje
data['Inflacion'] = pd.to_numeric(data['Inflacion'], errors='coerce') / 100

# Filtramos los datos desde 2000
data = data[data.index >= '2000-01-01']
display(data.head())

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(data['Inflacion'])
plt.title('Inflacion a traves del tiempo')
plt.xlabel('Fecha')
plt.ylabel('Inflacion')
plt.grid(True)
plt.show()

Vamos a desarrollar modelos que tomen doce meses e intenten predecir el decimo tercer mes. Por ejemplo, para predecir la inflación a la fecha de diciembre de 2025 el modelo toma los meses de diciembre de 2024, enero, febreo, marzo, abril, mayo, junio, julio, agosto, septiembre, octubre y noviembre de 2025.

### Ejercicio 1 - Secuencia de ventanas
Para entrenar un modelo recurrente es necesario transformar la serie temporal en un conjunto de secuencias de entrada y salida.

Este proceso se conoce como **sliding window** o ventana deslizante, y consiste en recorrer los datos históricos con una ventana de longitud fija que captura una secuencia de valores pasados (entrada), asociándola con uno o varios valores futuros (salida).

Por ejemplo, si tenemos la siguiente serie temporal:

$$ [x_1,x_2,\dotsb, x_t  ]$$

y tenemos una ventana ```n_steps_in = 3``` para retornar un solo valor ```n_steps_out = 1```. Entonces las muestras de entrenamiento son de la forma:

* Entrada: $[x_1,x_2,x_3]$. Salida: $x_4$
* Entrada: $[x_2,x_3,x_4]$. Salida: $x_5$.
* Entrada: $[x_{i-1},x_i,x_{i+1}]$. Salida: $x_{i+2}$.

**Instrucciones**:
-  Fija el número de iteraciones para aplicar la serie deslizante.
-  Agrega al arreglo de las secuencias de entrada `X` los datos desde el instante actual `i` hasta el número de entradas `i + n_steps_in`.
-  Agrega al arreglo de valores objetivo `Y` los datos desde el número de entradas `i + n_steps_in` hasta el número de salidas `i + n_steps_in + n_steps_out`.

In [ ]:
# Función calificadora: create_sequences

def create_sequences(data, n_steps_in, n_steps_out):
  """
  Crea secuencias de entrada-salida para RNN usando sliding window
  Argumentos:
  data -- array 1D con los datos de inflación
  n_steps_in -- número de pasos temporales de entrada (ventana)
  n_steps_out -- número de pasos a predecir (tamaño de la salida).

  Retorna:
  X -- array con secuencias de entrada, shape (n_samples, n_steps_in)
  Y -- array con valores objetivo, shape (n_samples, n_steps_out)
  """
  X, Y = [], []
  ### START CODE HERE ###
  # Recorremos la serie aplicando la ventana deslizante
  for i in range( ??? ):
    # Secuencia de entrada: ventana de n_steps_in valores. X.append(...)
    X.append( ??? )
    # Valor objetivo: el siguiente valor después de la ventana. Y.append(...)
    Y.append( ??? )
  ### END CODE HERE ###

  return np.array(X), np.array(Y)

In [ ]:
create_sequences_test(create_sequences)

**Datos esperados:**

Primer secuencia de entrada (X):
```
[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864]
```

Primer secuencia de salida (Y):
```
[0.15599452 0.05808361]
```

### Ejercicio 2 - Forma de los datos
La entrada ```x``` de los estados ocultos de nuestro modelo tienen la forma ```(n_x, m, T_x)```, que corresponde al número de neuronas, al número de muestras y a la longitud de la secuencia. Mientras que la salida ```y``` debe tener la forma ```(n_y,m,1)```, que corresponde al número de salidas, al tamaño de la muestra y a la única predicción que vamos a realizar.

**Instrucciones**:
-  Cambia la forma de los datos de entrada `X`y `Y` en las variables `X_reshaped` y `Y_reshaped`, respectivamente.

#### Pista:
-  Puede ser útil revisar la función ```reshape()``` en [NumPy](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html).


In [ ]:
# Función calificadora: reshape_data

def reshape_data(X, Y):
  """
  Transforma datos al formato requerido para la LSTM y la red GRU.
  Los modelos esperan:
  - X: shape (n_x, m, T_x) donde:
    n_x = número de features
    m = batch size (número de muestras)
    T_x = longitud de secuencia temporal
  - Y: shape (n_y, m, 1) donde:
    n_y = número de salidas
    m = batch size

  Argumentos:
  X -- array shape (m, T_x)
  Y -- array shape (m, n_steps_out)

  Retorna:
  X_reshaped -- array shape (n_x, m, T_x)
  Y_reshaped -- array shape (n_y, m, 1)
  """
  ### START CODE HERE ###
  X_reshaped =   # X: de (m, T_x) a (1, m, T_x)
  Y_reshaped =   # Y: de (m, n_steps_out) a (n_steps_out, m, 1)
  ### END CODE HERE ###

  return X_reshaped, Y_reshaped

In [ ]:
reshape_data_test(reshape_data)

### Preparación del conjunto de entrenamiento/prueba
Primero normalizamos los datos. Posteriormente creamos las secuencias con la función definida anteriormente.

In [ ]:
# Utilizamos normalización con escala estándar
scaler = StandardScaler()
inflation_scaled = scaler.fit_transform(data[['Inflacion']].values)
# Configuración de ventana temporal
n_steps_in = 12  # Usar 12 meses previos para predecir el siguiente
n_steps_out = 1  # Predecir 1 mes adelante

X_seq, y_seq = create_sequences(inflation_scaled, n_steps_in, n_steps_out)

Ahora dividimos el conjunto de secuencias en 80% para entrenamiento y el 20% para prueba.
Además, para entrenar los modelos con Keras, necesitamos mantener la misma forma.

In [ ]:
# Entrenamiento con el dataset
train_size = int(len(X_seq) * 0.8)  # 80% entrenamiento, 20% test

X_train_keras = X_seq[:train_size]
y_train_keras = y_seq[:train_size]
X_test_keras = X_seq[train_size:]
y_test_keras = y_seq[train_size:]

Finalmente cambiamos la forma de los datos para el entrenamiento para los modelos hechos en NumPy.

In [ ]:
X_train, y_train = reshape_data(X_train_keras, y_train_keras)
X_test, y_test = reshape_data(X_test_keras, y_test_keras)

<a name="a"></a>
## Problemas de gradientes en redes neuronales recurrentes

Sabemos que una red neuronal recurrente se define como una composición de funciones no lineales, usualmente $\tanh$ y $\sigma$ como se observa en las siguientes ecuaciones.

$$ \hat{y} = \tanh \left(W_y h^{\langle t\rangle} + b_y  \right) $$
$$ h^{\langle t\rangle} = \sigma \left( W_h \left[h^{\langle t-1\rangle}; x^{\langle t\rangle}  \right] + b_h  \right) $$

Si en algún paso $h^{\langle t\rangle}$ los cómputos de la retropropagación se hacen muy pequeños entonces se corre el riesgo de propagar este error a través de la composición, esta propagación de error hace que en algún momento el valor de los parámetros convergan al $0$ numérico, matando las neuronas.

Para contrarrestar estos problemas numéricos, las arquitecturas LSTM y GRU operan los valores de los estados ocultos con *compuertas* que determinan los valores a propagar a través de la red.

<a name="3"></a>
## Primer Modelo: Construyendo una LSTM

La arquitectura de la celda LSTM puede verse en la siguiente Figura.
<center>
<img src="https://raw.githubusercontent.com/MateoOrtiz001/Laboratory-LSTM-GRU-Inflation/main/imagenes/LSTM01.png" width="500">
</center>
Se tienen tres compuertas $\Gamma_f$,$\Gamma_u$ y $\Gamma_o$, una célula de memoria para el estado $c^{\langle t\rangle}$ y un candidato a dicha célula $\tilde{c}^{\langle t\rangle}$.

-  *Compuerta de olvido* $\Gamma_f$: es un tensor que contiene valores entre $0$ y $1$.
    - Si una unidad en esta compuerta está muy cerca del 0 entonces la LSTM "olvidará" el estado guardado en la correspondiente unidad de la célula de memoria previa.
    - Si una unidad en esta compuerta está muy cerca del 1 entonces la LSTM recordará fuertemente su correspondiente valor guardado en la unidad de la célula de memoria.

    $$\Gamma_f = \sigma \left(W_f \left[h^{\langle t-1\rangle}; x^{\langle t\rangle}\right] + b_f\right)  \tag{1}$$

-  *Célula de memoria candidata* $\tilde{c}^{\langle t\rangle}$: es un tensor que contiene información del estado de tiempo actual que podría ser guardado en la célula de memoria actual $c^{\langle t\rangle}$.
    - Como los datos están normalizados, este tensor contiene valores entre $-1$ y $1$.

    $$\tilde{c}^{\langle t\rangle} = \tanh\left(W_c \left[h^{\langle t-1\rangle}; x^{\langle t\rangle}\right] + b_c\right) \tag{2} $$

-  *Compuerta de actualización* $\Gamma_u$: es un tensor que contiene valores entre $0$ y $1$.
    - Esta compuerta decide qué valores del candidato $\tilde{c}^{\langle t\rangle}$ son pasados a la célula de memoria $c^{\langle t\rangle}$. Las unidades cercanas al 0 no dejarán pasar valores, mientras que las más cercanas al 1 dejarán pasar fuertemente los valores.

    $$\Gamma_u = \sigma \left(W_u \left[h^{\langle t-1\rangle}; x^{\langle t\rangle}\right] + b_u\right)  \tag{3}$$

-  *Célula de memoria* $c^{\langle t\rangle}$: como su nombre indica, es la "memoria" que será pasada a estados ocultos posteriores.
    - Es simplemente la combinación entre la célula de memoria del estado anterior y el valor candidato.

    $$c^{\langle t\rangle} = \Gamma_f^{\langle t\rangle} * c^{\langle t-1\rangle} + \Gamma_u^{\langle t\rangle} * \tilde{c}^{\langle t\rangle} \tag{4}$$

-  *Compuerta de salida* $\Gamma_o$: es un tensor que contiene valores entre $0$ y $1$.
    - Esta compuerta decide qué enviar a la salida del estado oculto.

    $$\Gamma_o = \sigma \left(W_o \left[h^{\langle t-1\rangle}; x^{\langle t\rangle}\right] + b_o\right)  \tag{5}$$

-  *Estado oculto* $h^{\langle t\rangle}$: es un tensor que sirve como entrada para el siguiente paso de tiempo.
    - Es usado para determinar las compuertas del siguiente paso del tiempo.
    - Es simplemente la combinación entre la compuerta de salida $\Gamma_o$ y el estado de memoria $c\langle t\rangle$, este último sobre la función $\tanh$.
    - La compuerta actúa para determinar qué valores de la célula de memoria deben ser incluídos en el estado oculto actual.

    $$h^{\langle t\rangle} = \Gamma_o * \tanh\left(c^{\langle t\rangle}\right) \tag{6}$$

<a name="3-1"></a>
### Ejercicio 3 - Celda del Foward Pass
Implementa la celda LSTM observada en la Figura.

**Intrucciones**:
1. Concatena el estado oculto $h^{\langle t-1 \rangle}$ $x^{\langle t \rangle}$ en una sola matriz:  

$$concat = \begin{bmatrix} h^{\langle t-1 \rangle} \\ x^{\langle t \rangle} \end{bmatrix}$$  

2. Computa todas las fórmulas (1 hasta 6) para las compuertas, el estado oculto y el estado de la célula.
3. Para este problema, no tienes que computar la predicción $y^{\langle t \rangle}$ en todas las celdas, luego no deberás implementarla aquí.

#### Pistas
* Puedes usar [numpy.concatenate](https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html).  Revisa qué valor usar para el parámetro `axis`.
* Las funciones `sigmoid()` y `softmax` se importan desde `utils.py`.
* La documentación sobre [numpy.tanh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tanh.html)
* Usa [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) para la multiplicación de matrices.

In [ ]:
# Función adaptada del curso de deepLearningAi. Lab15a: Building a Recurrent Neuronal Network Step by Step
# Función calificadora: lstm_cell_forward

def lstm_cell_forward(xt, h_prev, c_prev, parameters):
    """
    Implementar un solo paso de la celda LSTM como se describe en la Figura (4)
    Argumentos:
    xt -- la entrada en el timestep "t", numpy array de forma (n_x, m)
    h_prev -- Estado oculto en el tiempo "t-1", numpy array de la forma (n_a,m)
    c_prev -- Celula de memoria en el tiempo "t-1", numpy array of shape (n_a, m)
    parametros -- diccionario de python que contiene:
                        Wf -- Matriz de pesos de la compuerta de olvido, numpy array de forma (n_h, n_h + n_x)
                        bf -- Sesgo de la compuerta de olvido, numpy array de forma (n_a, 1)
                        Wu -- Matriz de pesos de la compuerta de actualización, numpy array de forma (n_h, n_h + n_x)
                        bu -- Sesgo de la compuerta de actualización, numpy array de forma (n_a, 1)
                        Wc -- Matriz de pesos de la primera "tanh", numpy array de forma (n_h, n_h + n_x)
                        bc --  Sesgo de la primera "tanh", numpy array de forma (n_h, 1)
                        Wo -- Matriz de pesos de la compuerta de salida, numpy array de forma (n_h, n_h + n_x)
                        bo --  Sesgo de la compuerta de salida, numpy array de forma (n_h, 1)


    Returns:
    h_next -- siguiente estado oculto, de forma (n_a, m)
    c_next -- siguiente estado de memoria, de forma (n_a, m)
    cache -- tupla de valores necesarios para el paso hacia atrás, contiene (h_next, c_next, h_prev, c_prev, xt, parameters)

    Nota: ft/ut/ot significan las compuertas de olvido/actualización/salida, cct significa el valor candidato (c tilde),
          c significa el estado de la celda (memoria)
    """

    # Recuperar parámetros de "parameters"
    Wf = parameters["Wf"] # peso de la compuerta de olvido
    bf = parameters["bf"]
    Wu = parameters["Wu"] # peso de la compuerta de actualización (observar el nombre de la variable)
    bu = parameters["bu"] # (observar el nombre de la variable)
    Wc = parameters["Wc"] # peso del valor candidato
    bc = parameters["bc"]
    Wo = parameters["Wo"] # peso de la compuerta de salida
    bo = parameters["bo"]


    # Recuperar dimensiones de las formas de xt
    n_x, m = xt.shape
    n_a, _ = h_prev.shape # Asumiendo que la forma de h_prev es (n_a, m)

    ### START CODE HERE ###
    # Concatenar h_prev y xt
    concat =

    # Calcular valores para ft, ut, cct, c_next, ot, h_next usando las fórmulas dadas en la figura (4)
    ft =
    ut =
    cct =
    c_next =
    ot =
    h_next =

    ### END CODE HERE ###

    # almacenar valores necesarios para la propagación hacia atrás en la caché
    cache = (h_next, c_next, h_prev, c_prev, ft, ut, cct, ot, xt, parameters)

    return h_next, c_next, cache

In [ ]:
lstm_cell_forward_test(lstm_cell_forward)

<a name="3-2"></a>
### Ejercicio 4 - Forward Pass

Ya haz implementado un paso de una LSTM, ahora puedes iterar sobre toda la secuencia utilizando un bucle sobre todas las entradas $T_x$.

Implementa el modelo LSTM iterando las celdas hasta los $T_x$ pasos de tiempo.

**Intrucciones**:
-  Obtén las dimensiones $n_x,n_h,n_y,m,T_x$ de las formas de las variables `x`y `parameters`.
-  Inicializa los tensores 3D $h,c$ y $y$
    -  $h$: estado oculto, forma $(n_h,m,T_x)$.
    -  $c$: célula de memoria, forma $(n_h,m,T_x)$.
    -  $y$: predicción, forma $(n_h,m,T_y)$ donde $T_y=1$.
-  Inicializa el tensor 2D $h^{\langle t\rangle}$.
    -  $h^{\langle t\rangle}$ guarda el estado oculto para el paso de tiempo $t$. El nombre de la variable es `h_next`.
    -  $h^{\langle 0\rangle}$, el estado oculto en el tiempo $0$, se pasa cuando se llama la función. El nombre de la variable es `h0`.
    -  Inicializa $h^{\langle t\rangle}$ con el estado oculto $h^{\langle 0\rangle}$ que se pasa a la función.
-  Inicializa $c^{\langle t\rangle}$ con ceros.
    -  El nombre de la variable es `c_next`.
-  Para cada instante de tiempo, realiza lo siguiente
    -  Desde el tensor 3D $x$, toma la parte 2D $x^{\langle t\rangle}$ en el instante de tiempo $t$.
    -  Llama la función `lstm_cell_forward` definida anteriormente, para obtener el estado oculto, la célula de memoria y el cache.
    -  Guarda el estado oculto y la célula de memoria dentro de los tensores 3D.
    -  Agrega el cache a los caches.

*Nota*: En este caso observa que la predicción se retorna directamente como un tensor 2D.

In [ ]:
# Función adaptada del curso de deepLearningAi. Lab15a: Building a Recurrent Neuronal Network Step by Step
# Función calificadora: lstm_forward

def lstm_forward(x, h0, parameters):
    """
    Implementa la propagación hacia adelante de la red neuronal recurrente usando una celda LSTM descrita en la Figura (4).

    Argumentos:
    x -- Datos de entrada para cada paso de tiempo, de forma (n_x, m, T_x).
    h0 -- Estado oculto inicial, de forma (n_h, m)
    parameters -- diccionario de python que contiene:
                        Wf -- Matriz de pesos de la compuerta de olvido, numpy array de forma (n_h, n_h + n_x)
                        bf -- Sesgo de la compuerta de olvido, numpy array de forma (n_h, 1)
                        Wu -- Matriz de pesos de la compuerta de actualización, numpy array de forma (n_h, n_h + n_x)
                        bu -- Sesgo de la compuerta de actualización, numpy array de forma (n_h, 1)
                        Wc -- Matriz de pesos de la primera "tanh", numpy array de forma (n_h, n_h + n_x)
                        bc -- Sesgo de la primera "tanh", numpy array de forma (n_h, 1)
                        Wo -- Matriz de pesos de la compuerta de salida, numpy array de forma (n_h, n_h + n_x)
                        bo -- Sesgo de la compuerta de salida, numpy array de forma (n_h, 1)
                        Wy -- Matriz de pesos que relaciona el estado oculto con la salida, numpy array de forma (n_y, n_h)
                        by -- Sesgo que relaciona el estado oculto con la salida, numpy array de forma (n_y, 1)

    Returns:
    h -- Estados ocultos para cada paso de tiempo, numpy array de forma (n_h, m, T_x)
    y -- Predicciones para cada paso de tiempo, numpy array de forma (n_y, m)
    c -- El valor del estado de la celda, numpy array de forma (n_h, m, T_x)
    caches -- tupla de valores necesarios para el paso hacia atrás, contiene (lista de todas las cachés, x)
    """

    # Inicializar "caches", que rastreará la lista de todas las cachés
    caches = []
    Wy = parameters['Wy']
    by = parameters['by']

    ### START CODE HERE ###
    # Recuperar dimensiones de las formas de x y parameters['Wy']
    n_x, m, T_x =
    n_y, n_h =

    # inicializar "h", "c" y "y" con ceros
    h =
    c =
    y =

    # Inicializar h_next y c_next
    h_next =
    c_next =

    # iterar sobre todos los pasos de tiempo
    for t in range(T_x): # Iterar a través de todos los pasos de tiempo
        # Obtener la sección 2D 'xt' de la entrada 3D 'x' en el paso de tiempo 't'
        xt = x[:,:,t]
        # Actualizar el siguiente estado oculto, el siguiente estado de memoria, calcular la predicción, obtener la caché
        h_next, c_next, cache =

        # Guardar el valor del nuevo estado oculto "siguiente" en h
        h[:,:,t] =
        # Guardar el valor del siguiente estado de celda
        c[:,:,t]  =
        # Añadir la caché a caches
        caches.append( ??? )
    ### END CODE HERE ###

    y = np.dot(Wy,h[:,:,-1]) + by

    # almacenar valores necesarios para la propagación hacia atrás en la caché
    caches = (caches, x)

    return h, y, c, caches

In [ ]:
lstm_forward_test(lstm_forward)

<a name="3-3"></a>
### Ejercicio 5 - Celda del Backward Pass

Ya con la propagación hacia adelante programada, ahora podemos calcular le retropropagación hacia atrás para el entrenamiento del modelo.
<center>
<img src="https://raw.githubusercontent.com/MateoOrtiz001/Laboratory-LSTM-GRU-Inflation/main/imagenes/LSTM02.png" width="500">
</center>
En cada celda calculamos la derivada de las variables asociadas a los pesos y sesgos de las compuertas, y las derivadas de la entrada $x^{\langle t\rangle}$ y de los estados anteriores $h^{\langle t-1\rangle}$ y $c^{\langle t-1\rangle}$.

Como la retropropagación se hace hacia atrás de forma iterativa desde el instante $T_x$, en cada paso tenemos $dh^{\langle t\rangle}$ y $dc^{\langle t\rangle}$, llamados en el código `dh_next` y `dc_next` respectivamente. Con esto, las derivadas de cada compuerta está dada como sigue.

\begin{align}
do^{\langle t \rangle} &= dh^{\langle t \rangle}*\tanh(c^{\langle t\rangle}) * \Gamma_o^{\langle t \rangle}*\left(1-\Gamma_o^{\langle t \rangle}\right)\tag{1} \\[8pt]
d\widetilde{c}^{\langle t \rangle} &= \left(dc^{\langle t\rangle}*\Gamma_u^{\langle t \rangle}+ \Gamma_o^{\langle t \rangle}* (1-\tanh^2(c^{\langle t\rangle})) * \Gamma_u^{\langle t \rangle} * dh^{\langle t \rangle} \right) * \left(1-\left(\widetilde c^{\langle t \rangle}\right)^2\right) \tag{2} \\[8pt]
du^{\langle t \rangle} &= \left(dc^{\langle t\rangle}*\widetilde{c}^{\langle t \rangle} + \Gamma_o^{\langle t \rangle}* (1-\tanh^2(c^{\langle t\rangle})) * \widetilde{c}^{\langle t \rangle} * dh^{\langle t \rangle}\right)*\Gamma_u^{\langle t \rangle}*\left(1-\Gamma_u^{\langle t \rangle}\right)\tag{3} \\[8pt]
df^{\langle t \rangle} &= \left(dc^{\langle t\rangle}* c^{\langle t-1\rangle} + \Gamma_o^{\langle t \rangle} * (1-\tanh^2(c^{\langle t\rangle}) * c^{\langle t-1\rangle} * dh^{\langle t \rangle}\right)*\Gamma_f^{\langle t \rangle}*\left(1-\Gamma_f^{\langle t \rangle}\right)\tag{4}
\end{align}

Con esto podemos calcular las derivadas de los parámetros. Primero, la derivada de los pesos está dada como

$ dW_f = df^{\langle t \rangle} \begin{bmatrix} h^{\langle t-1\rangle} \\ x^{\langle t\rangle}\end{bmatrix}^T \tag{5} $
$ dW_u = du^{\langle t \rangle} \begin{bmatrix} h^{\langle t-1\rangle} \\ x^{\langle t\rangle}\end{bmatrix}^T \tag{6} $
$ dW_c = d\widetilde c^{\langle t \rangle} \begin{bmatrix} h^{\langle t-1\rangle} \\ x^{\langle t\rangle}\end{bmatrix}^T \tag{7} $
$ dW_o = do^{\langle t \rangle} \begin{bmatrix} h^{\langle t-1\rangle} \\ x^{\langle t\rangle}\end{bmatrix}^T \tag{8}$

Para calcular los sesgos $db_f, db_u, db_c, db_o$ solo necesitas sumar a través de todas las 'm' muestras (axis= 1) de $df^{\langle t \rangle}, du_u^{\langle t \rangle}, d\widetilde c^{\langle t \rangle}, d_o^{\langle t \rangle}$ respectivamente. Observa que tienes que tener `keepdims = True`.

$\displaystyle db_f = \sum_{batch}df^{\langle t \rangle}\tag{9}$
$\displaystyle db_u = \sum_{batch}du^{\langle t \rangle}\tag{10}$
$\displaystyle db_c = \sum_{batch}dc^{\langle t \rangle}\tag{11}$
$\displaystyle db_o = \sum_{batch}do^{\langle t \rangle}\tag{12}$

Finalmente, debes calcular la derivada con respeceto al estado anterior, la célula de memoria anterior y la salida.
$ dh^{\langle t-1\rangle} = W_f^T df^{\langle t \rangle} + W_u^T   du^{\langle t \rangle}+ W_c^T d\widetilde c^{\langle t \rangle} + W_o^T do^{\langle t \rangle} \tag{13}$

Aquí, para referirnos a la concatenación de los pesos para la ecuación (13) son los primeros $n_h$ valores, es decir, $W_f = W_f[:,:n_h]$, etc.

$ dc^{\langle t-1\rangle} = dc^{\langle t\rangle}*\Gamma_f^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} * (1- \tanh^2(c^{\langle t\rangle}))*\Gamma_f^{\langle t \rangle}*dh^{\langle t\rangle} \tag{14}$

$ dx^{\langle t \rangle} = W_f^T df^{\langle t \rangle} + W_u^T  du^{\langle t \rangle}+ W_c^T d\widetilde c^{\langle t \rangle} + W_o^T do^{\langle t \rangle}\tag{15} $

en donde los pesos para la ecuación (15) van desde $n_a$ hasta el final, es decir $W_f = W_f[:,n_a:]$, etc.

**Instrucciones:**
-  Implementa las ecuaciones (1) - (15) de arriba.
    - $do^{\langle t \rangle}$ es representado por  `dot`,    
    - $d\widetilde{c}^{\langle t \rangle}$ es representado por  `dcct`,  
    - $du^{\langle t \rangle}$ es representado por  `dot`,  
    - $df^{\langle t \rangle}$ es representado por `dft`.

In [ ]:
# Función adaptada del curso de deepLearningAi. Lab15a: Building a Recurrent Neuronal Network Step by Step
# Función calificadora: lstm_cell_backward

def lstm_cell_backward(dh_next, dc_next, cache):
    """
    Implementa el paso hacia atrás para la celda LSTM (un solo paso de tiempo).

    Argumentos:
    dh_next -- Gradientes del siguiente estado oculto, de forma (n_a, m)
    dc_next -- Gradientes del siguiente estado de celda, de forma (n_a, m)
    cache -- caché que almacena información del paso hacia adelante

    Returns:
    gradients -- diccionario de python que contiene:
                        dxt -- Gradiente de los datos de entrada en el paso de tiempo t, de forma (n_x, m)
                        dh_prev -- Gradiente con respecto al estado oculto anterior, numpy array de forma (n_a, m)
                        dc_prev -- Gradiente con respecto al estado de memoria anterior, de forma (n_a, m, T_x)
                        dWf -- Gradiente con respecto a la matriz de pesos de la compuerta de olvido, numpy array de forma (n_a, n_a + n_x)
                        dWu -- Gradiente con respecto a la matriz de pesos de la compuerta de actualización, numpy array de forma (n_a, n_a + n_x)
                        dWc -- Gradiente con respecto a la matriz de pesos de la compuerta de memoria, numpy array de forma (n_a, n_a + n_x)
                        dWo -- Gradiente con respecto a la matriz de pesos de la compuerta de salida, numpy array de forma (n_a, n_a + n_x)
                        dbf -- Gradiente con respecto a los sesgos de la compuerta de olvido, de forma (n_a, 1)
                        dbu -- Gradiente con respecto a los sesgos de la compuerta de actualización, de forma (n_a, 1)
                        dbc -- Gradiente con respecto a los sesgos de la compuerta de memoria, de forma (n_a, 1)
                        dbo -- Gradiente con respecto a los sesgos de la compuerta de salida, de forma (n_a, 1)
    """

    # Recuperar información de "cache"
    (h_next, c_next, h_prev, c_prev, ft, ut, cct, ot, xt, parameters) = cache

    ### START CODE HERE ###
    # Recuperar dimensiones de las formas de xt y h_next
    n_x, m =
    n_h, m =

    # Calcular derivadas relacionadas con las compuertas. Sus valores se pueden encontrar observando cuidadosamente las ecuaciones (1) a (4)
    dot =
    dcct =
    dut =
    dft =

    # Calcular derivadas relacionadas con los parámetros. Usar ecuaciones (11)-(18)
    concat_t =
    dWf =
    dWu =
    dWc =
    dWo =
    dbf =
    dbu =
    dbc =
    dbo =

    # Calcular derivadas con respecto al estado oculto anterior, estado de memoria anterior y entrada. Usar ecuaciones (19)-(21).
    dh_prev =
    dc_prev =
    dxt =
    ### END CODE HERE ###

    # Guardar gradientes en el diccionario
    gradients = {"dxt": dxt, "dh_prev": dh_prev, "dc_prev": dc_prev, "dWf": dWf,"dbf": dbf, "dWu": dWu,"dbu": dbu,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}

    return gradients

In [ ]:
lstm_cell_backward_test(lstm_cell_backward,lstm_cell_forward)

<a name="3-4"></a>
### Ejercicio 6 - Backward Pass

Calcular los gradientes del costo con respecto a $h^{\langle t\rangle}$ en cada paso de tiempo $t$ es útil porque ayuda a retropropagar el gradiente a la celda LSTM anterior. Para hacerlo, tienes que iterar a través de todos los pasos de tiempo empezando por el final $T_x$, y en cada paso, incrementar las derivadas de los parámetros y guardar $dx$.

**Instrucciones**:
-  Crea un bucle empezando desde $T_x$ retrocediendo. Para cada paso, llama tu función `lst_cell_backward` y actualiza los antiguos gradientes sumando los nuevos gradientes.
    -  Observa que `dxt`no se actualiza, pero sí se guarda.

In [ ]:
# Función adaptada del curso de deepLearningAi. Lab15a: Building a Recurrent Neuronal Network Step by Step
# Función calificadora: lstm_backward

def lstm_backward(dh, caches, m_batch):

    """
    Implementa el paso hacia atrás para la RNN con celda LSTM (sobre una secuencia completa).

    Argumentos:
    dh -- Gradientes con respecto a los estados ocultos, numpy-array de forma (n_h, m, T_x)
    caches -- caché que almacena información del paso hacia adelante (lstm_forward)
    m_batch -- El tamaño del minibatch, entero

    Returns:
    gradients -- diccionario de python que contiene:
                        dx -- Gradiente de las entradas, de forma (n_x, m, T_x)
                        dh0 -- Gradiente con respecto al estado oculto anterior, numpy array de forma (n_h, m)
                        dWf -- Gradiente con respecto a la matriz de pesos de la compuerta de olvido, numpy array de forma (n_h, n_h + n_x)
                        dWu -- Gradiente con respecto a la matriz de pesos de la compuerta de actualización, numpy array de forma (n_h, n_h + n_x)
                        dWc -- Gradiente con respecto a la matriz de pesos de la compuerta de memoria, numpy array de forma (n_h, n_h + n_x)
                        dWo -- Gradiente con respecto a la matriz de pesos de la compuerta de salida, numpy array de forma (n_h, n_h + n_x)
                        dbf -- Gradiente con respecto a los sesgos de la compuerta de olvido, de forma (n_h, 1)
                        dbu -- Gradiente con respecto a los sesgos de la compuerta de actualización, de forma (n_h, 1)
                        dbc -- Gradiente con respecto a los sesgos de la compuerta de memoria, de forma (n_h, 1)
                        dbo -- Gradiente con respecto a los sesgos de la compuerta de salida, de forma (n_h, 1)
    """

    # Recuperar valores de la primera caché de caches.
    (caches, x) = caches
    (h1, c1, h0, c0, f1, u1, cc1, o1, x1, parameters) = caches[0]

    ### START CODE HERE ###
    # Recuperar dimensiones de las formas de dh y x1
    n_h, m, T_x =
    n_x, m =

    # inicializar los gradientes con los tamaños correctos
    dx =
    dh0 =
    dh_prevt =
    dc_prevt =
    dWf =
    dWu =
    dWc =
    dWo =
    dbf =
    dbu =
    dbc =
    dbo =

    # iterar hacia atrás sobre toda la secuencia
    for t in reversed(range(T_x)):
        # Calcular todos los gradientes usando lstm_cell_backward.
        gradients =
        # Almacenar o añadir el gradiente al gradiente del paso anterior de los parámetros
        dh_prevt =
        dc_prevt =
        dx[:,:,t] =
        dWf +=
        dWu +=
        dWc +=
        dWo +=
        dbf +=
        dbu +=
        dbc +=
        dbo +=
    # Establecer el gradiente de la primera activación al gradiente retropropagado dh_prev.
    dh0 =

    ### END CODE HERE ###
    dWf /= m_batch; dWu /= m_batch; dWc /= m_batch; dWo /= m_batch
    dbf /= m_batch; dbu /= m_batch; dbc /= m_batch; dbo /= m_batch
    # Almacenar los gradientes en un diccionario de python
    gradients = {"dx": dx, "dh0": dh0, "dWf": dWf,"dbf": dbf, "dWu": dWu,"dbu": dbu,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}

    return gradients

In [ ]:
lstm_backward_test(lstm_backward,lstm_cell_backward,lstm_cell_forward)

### Ejercicio 7 - Inicialización de parámetros

Para la inicialización de los parámetros necesitamos obsevar que para $W_u,W_f,W_o$ y $W_c$ tenemos como función de activación la función $\tanh$ y para la salida $W_y$ se activa con la función $\sigma$. Esto con el objetivo de evitar problemas numéricos como la explosión o el desvanecimiento de gradientes.

Para ambas funciones vamos a considerar la inicialización *Xavier* (1). Esto corresponde a tomar una distribución Gausiana con media $0$ y desviación estándar

$$ \sigma = \sqrt{\frac{2}{N}}\tag{1} $$

en donde $N$ es la dimensión de entrada de cada matriz de pesos $W$.

Mientras que para los bias $b$ inicializaremos con vectores en blanco.

**Instrucciones**:
-  Inicializa las matrices de pesos de acuerdo a la inicialización *Xavier Glorot*.
    -  Los parámetros `W` de las compuertas tienen forma `(n_h, n_h + n_x)`.
    -  El parámetro `Wy` tiene la forma `(n_y, n_h)`.
-  Inicializa los vectores de sesgo con ceros.
    - Los parámetros `b` de las compuertas tienen la forma `(n_h,1)`.
    - El parámetro `by` tiene la forma `(n_y,1)`.

#### Pistas:
-  Considera inicializar los pesos utilizando [np.random.randn](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html), que tiene distribución Gaussiana $\mathcal{N}(0,1)$, y a dicho tensor aplicarle el producto con la desviación estándar.

In [ ]:
# Función calificadora: initialize_LSTM_parameters

def initialize_LSTM_parameters(n_h, n_x, n_y):
  """
  Inicializa los parámetros de la LSTM con Xavier initialization,
  utilizando la fórmula (1)

  Arguments:
  n_a -- número de unidades en el hidden state
  n_x -- número de features de entrada
  n_y -- número de features de salida

  Returns:
  parameters -- diccionario con los parámetros inicializados
  """
  np.random.seed(1)

  parameters = {}
  ### START CODE HERE ###

  parameters['Wf'] =
  parameters['bf'] =

  parameters['Wu'] =
  parameters['bu'] =

  parameters['Wc'] =
  parameters['bc'] =

  parameters['Wo'] =
  parameters['bo'] =

  parameters['Wy'] =
  parameters['by'] =

  ### END CODE HERE ###
  return parameters


In [ ]:
initialize_LSTM_parameters_test(initialize_LSTM_parameters)

<a name="3-5"></a>
### Ejercicio 8 - Entrenamiento del modelo LSTM
Ya tienes el modelo para hacer propagación hacia adelante y retropropagación hacia atrás. Ahora, con la inicialización de parámetros, es momento de implemenetar el entrenamiento del modelo.

En esta implementación vamos a utilizar el algoritmo de adam como optimizador con mini lotes -no te preocupes de esto, pues ya viene implementado y tu solo deberás programar los cálculos de la propagación y la retropropagación- y MSE como función de costo.

Para este ejercicio es importante saber que nuestra función de costo se define

$$\mathcal{L} = \frac{1}{2m_{batch}} \sum_{i\in batch}\left(y_{pred} - y_{real}\right)^2 \tag{1}$$

así, la derivada de esta función con respecto a la predicción $y_{pred}$ es de la forma

$$d\hat y =\frac{d\mathcal L}{d\hat y} = \frac{1}{m_{batch}} \sum_{i\in batch}\left( y_{pred} - y_{real} \right) \tag{2}$$

Con esto podemos calcular los parámetros de la salida y los valores del último paso del estado oculto $h^{\langle T_x\rangle}$ y de la célula de memoria $c^{\langle T_x\rangle}$. Sus derivadas son:

$ d h^{\langle T_x\rangle} = W_y^T d \hat y\tag{3}$


$ d c^{\langle T_x\rangle} = d \hat y * \Gamma_o^{\langle T_x\rangle} * \left(1 - \tanh^2\left(c^{\langle T_x \rangle}\right)\right) \tag{4}$

Y la derivada de los parámetros de $\hat y$ se calculan de forma similar a los parámetros de las compuertas:

$$ d W_y = d\hat y (h^{\langle t\rangle})^T \tag{5}$$
$$ d b_y = \sum_{batches} d \hat y \tag{6}$$

Como ahora manejamos lotes, el número de muestras está dado por `m_batch`.

**Instrucciones**:
-   Obtén los valores del estado oculto `h`, la predicción `y_pred`, célula de memoria `c` y el cache `cache`, calculando la propagación hacia adelante.
    - El estado oculto `h`tiene la forma `(n_h,m_batch,T_x)`.
    - La predicción `y_pred` tiene la forma `(n_y,m_batch)`.
    - La célula de memoria `c`tiene la forma `(n_h,m_batch,T_x)`.
-   Calcula el error de predicción llamando la función `compute_cost()` con las variables `y_pred` y `minibatch_Y`.
    - La variable del mini lote `minibatch_Y` tiene forma `(n_y,m_batch,T_x)`.
-   Inicializa las derivadas $d h$ y $dc$ en las variables `dh` y `dc`, respectivamente.
    - Ambas variables tienen la forma `(n_h,m_batch,T_x)`.
-   Calcula la derivada de la función de costo utilizando la ecuación (2) en la variable `dcost_dy_pred`.
-   Calcula la derivada del estado oculto $h^{\langle T_x\rangle}$ y de la célula de memoria $c^{\langle T_x\rangle}$ utilizando las ecuaciones (3) y (4), respectivamente.
    -   Accede al paso $T_x$ con `h[:,:,-1]` y `c[:,:,-1]`.
    -   Los valores de la propagación se han extraído anteriormente.
-   Obtén los gradientes calculando la retropropagación.
-   Actualiza el valor de las derivadas de los parámetros $W_y$ y $b_y$ utilizando las ecuaciones (5) y (6).

#### Pistas:
-  La función [np.zeros_like](https://numpy.org/devdocs/reference/generated/numpy.zeros_like.html) puede ser útil para inicializar los tensores.
-  Utiliza la función [np.squeeze](https://numpy.org/devdocs/reference/generated/numpy.squeeze.html) para igualar dimensiones entre las variables `y_pred` y `minibatch_Y`.

In [ ]:
# Función calificadora: model_lstm_adam

def model_lstm_adam(X, Y, n_h, learning_rate=0.001, mini_batch_size=32,
                    beta1=0.9, beta2=0.999, epsilon=1e-8,
                    num_epochs=1000, print_cost=True):
  """
  Entrena una LSTM desde cero con Adam y minibatches.

  Arguments:
  X -- datos de entrada, numpy array (n_x, m, T_x)
  Y -- datos objetivo, numpy array (n_y, m, 1)
  n_a -- número de unidades del estado oculto
  learning_rate -- tasa de aprendizaje Adam
  mini_batch_size -- tamaño de minibatch
  num_epochs -- número de épocas
  """

  n_x, m, T_x = X.shape
  n_y, _, _ = Y.shape

  # Inicialización
  parameters = initialize_LSTM_parameters(n_h, n_x, n_y)
  v, s = initialize_adam(parameters)
  seed = 10
  costs = []
  t = 0  # contador de pasos de Adam

  for epoch in range(num_epochs):
    seed += 1
    minibatches = random_mini_batches(X, Y, mini_batch_size, seed)
    epoch_cost = 0

    for minibatch_X, minibatch_Y in minibatches:
      # Inicialización del estado oculto
      m_batch = minibatch_X.shape[1]
      h0 = np.zeros((n_h, m_batch))

      ### START CODE HERE ###
      # Forward pass utilizando tu función implementada anteriormente lstm_forward(minibatch_X, h0, parameters)
      h, y_pred, c, caches =

      # Obtenemos los valores del cache en el último instante
      caches_list = caches[0] if isinstance(caches, tuple) else caches
      _, c_next_last, _, _, _, _, _, ot_last, _, _ = caches_list[-1]

      # Costo (MSE) - Calcula y_pred (n_y, m) con minibatch_Y (n_y, m, 1), utiliza .squeeze(axis=2) para igualar dimensiones.
      cost =

      # Inicializa las derivadas del estado oculto y de la celula de memoria.
      dh = # Forma (n_h, m_batch, T_x)
      dc = # Forma (n_h, m_batch, T_x)

      # Calcula la derivada de la predicción de acuerdo a la ecuación (2). Recuerda utilizar .squeeze(axis=2) para igualar dimensiones.
      dcost_dy_pred =

      # Propaga la derivada al último bloque, utiliza las ecuaciones (3)-(4).
      dh[:, :, -1] =
      dc[:, :, -1] =

      # Backward pass utilizando tu función implementada anteriormente lstm_backward(dh, caches, m_batch).
      grads =

      # Calcula las derivadas de los parámetros de la salida, utiliza las ecuaciones (5)-(6).
      grads['dWy'] =
      grads['dby'] =

      ### END CODE HERE ###

      # Recortar gradientes
      grads = clip_gradients(grads, 5.0)
      # Update Adam
      t += 1
      parameters, v, s = update_parameters_with_adam(parameters, grads, v, s, t, learning_rate, beta1, beta2, epsilon)
      epoch_cost += cost / len(minibatches)

    # Guardar costo
    if print_cost:
      print(f"Cost after epoch {epoch}: {epoch_cost:.6f}")
      costs.append(epoch_cost)

  # Graficar costos
  if print_cost:
    plot_training(costs)


  return parameters

In [ ]:
model_lstm_adam_test(model_lstm_adam)

Ejecuta la siguiente celda para entrenar tu primer modelo LSTM. Puedes experimentar con los diversos hiperparámetros

In [ ]:
modeloLSTM = model_lstm_adam(
    X_train,
    y_train,
    n_h= 78,                # Número de unidades del estado oculto
    learning_rate=0.001,    # Tasa de aprendizaje
    mini_batch_size=64,     # Tamaño del minibatch
    num_epochs=200,         # Número de épocas
    print_cost=True         # Mostrar el costo
)


<a name="3-6"></a>
### Ejercicio 9 - Predicción del modelo LSTM
Listo, ya haz entrenado tu modelo LSTM. Sin embargo, ahora queremos hacer predicciones con este. Tu función `lstm_forward` realiza una sola predicción sobre el número $T_x$ de pasos. Sin embargo, si queremos realizar predicciones sobre más instantes de tiempo, debemos actualizar las ventanas de tiempo y los valores de las ventanas con los valores predichos.

Para esto nuestra función `predict_lstm` recibe como parámetros la ventana actual `window`, el modelo `parameters` y el número de pasos `steps`. Nosotros extraemos los valores de las dimensiones necesarios e inicializamos el estado oculto inicial $h^{\langle 0\rangle}$ y una lista con los valores a predecir `future_LSTM_predictions`. Tu objetivo es implementar el bucle principal.

**Instrucciones**:
-  Ajusta el número de iteraciones en el bucle principal.
-  Realiza la predicción $\hat y$ utilizando la función `lstm_forward`.
    - Recuerda que la función `lstm_forward` retorna una tupla `(h,y,c,caches)`, en este caso solo nos interesa el valor `y`.
-  Guarda la predicción en la lista `future_LSTM_predictions`.
    - Aplana el tensor 2D para guardar un solo valor.
-  Actualiza una nueva ventana con el valor que acabas de predecir $\hat y$.
-  Actualiza la ventana redimensionándo la nueva ventana para tener forma `(1, 1, T_x)`.

In [ ]:
# Función calificadora: predict_lstm

def predict_lstm(window, parameters, steps):
  """
  Realiza un número de predicciones con el modelo LSTM empezando desde una ventana dada.

  Argumentos:
  window -- Ventana de entrada inicial para la predicción, shape (n_x, 1, T_x)
  parameters -- Diccionario de Python con los parámetros entrenados (Wf, bf, Wu, bu, ...)
  steps -- Número de pasos de predicción a realizar

  Retorna:
  future_LSTM_predictions -- Una lista con las predicciones.
  """
  # Extraer dimensiones de la entrada.
  n_x, m, T_x = window.shape
  n_y, n_h = parameters["Wy"].shape

  # Lista para guardar las predicciones.
  future_LSTM_predictions_scaled = []
  # Inicializa el primer estado oculto con ceros.
  h0 = np.zeros((n_h, 1))

  ## START CODE HERE ##
  # Itera hasta el número de pasos dado en el parámetro steps.
  for _ in range( ??? ):
    # Realiza la prediccion sobre el conjunto de datos y de parámetros.
    h, y, c, caches =

    # Guarda la prediccion, aplana el tensor 2D utilizando .flatten().
    future_LSTM_predictions_scaled.append( ??? )

    # Actualiza los valores de la ventana. Aplana el tensor de la ventana primero.
    new_window_values = np.append( ??? )

    # Redimensiona la ventana actual para mantener la forma (1,1,T_x).
    window = new_window_values.reshape( ??? )
  ## END CODE HERE ##

  #Desnormaliza los datos
  future_LSTM_predictions = scaler.inverse_transform(np.array(future_LSTM_predictions_scaled).reshape(-1, 1))

  return future_LSTM_predictions.flatten()

In [ ]:
predict_lstm_test(predict_lstm)

### Predicción
Ahora veamos la predicción del modelo para los doce meses próximos del dataset.

In [ ]:
# Tomamos la última ventana de datos conocida del conjunto de prueba
current_window = X_test[:, -1:, :]
test_LSTM_prediction = predict_lstm(current_window, modeloLSTM, 24)

print("Predicciones de inflación para los próximos 24 meses:")
print(test_LSTM_prediction)

In [ ]:
# Des-normalizar los datos para la gráfica
y_train_actual = scaler.inverse_transform(y_train_keras.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test_keras.reshape(-1, 1))

# Crea un índice de fechas para la gráfica
# Los índices de las secuencias corresponden a la *última* fecha de cada ventana.
# Por lo tanto, y_train[i] corresponde a data.index[i + n_steps_in].
start_train_index = n_steps_in
end_train_index = start_train_index + train_size
train_dates = data.index[start_train_index:end_train_index]

start_test_index = end_train_index
end_test_index = start_test_index + len(y_test_keras)
test_dates = data.index[start_test_index:end_test_index]
# Crear fechas futuras para las predicciones
# La primera fecha futura es el mes siguiente a la última fecha de prueba
future_dates = pd.date_range(start=test_dates[-1], periods=25, freq='ME')[1:]
# Graficar los resultados
plt.figure(figsize=(15, 7))

# Datos originales
plt.plot(train_dates, y_train_actual, label='Datos de Entrenamiento (Reales)', color='blue')
plt.plot(test_dates, y_test_actual, label='Datos de Prueba (Reales)', color='green')

# Predicciones
# future_predictions ya está desnormalizado
plt.plot(future_dates, test_LSTM_prediction, label='Predicción a 24 Meses', color='red', linestyle='--')

# Estilo del gráfico
plt.title('Predicción de Inflación con LSTM', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valor de Inflación', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

<a name="4"></a>
## Segundo Modelo: Construyendo una GRU

La arquitectura de la celda GRU puede verse en la siguiente Figura.
<center>
<img src="https://raw.githubusercontent.com/MateoOrtiz001/Laboratory-LSTM-GRU-Inflation/main/imagenes/GRU01.png" width="500">
</center>
La GRU (Gated Recurrent Unit) es una versión simplificada de la LSTM. Se tienen dos compuertas $\Gamma_r$ y $\Gamma_u$, un estado oculto $h^{\langle t\rangle}$ y un candidato al estado oculto $\tilde{h}^{\langle t\rangle}$.

-  *Compuerta de actualización* $\Gamma_u$: es un tensor que contiene valores entre $0$ y $1$.
    - Determina qué tanto del estado oculto anterior debe ser conservado y qué tanto del nuevo candidato debe ser incorporado.
    - Valores cercanos a $0$ indican que se debe conservar principalmente el estado anterior.
    - Valores cercanos a $1$ indican que se debe incorporar principalmente el nuevo candidato.

    $$\Gamma_u = \sigma \left(W_u \left[h^{\langle t-1\rangle}; x^{\langle t\rangle}\right] + b_u\right)  \tag{1}$$

-  *Compuerta de relevancia* (o reset) $\Gamma_r$: es un tensor que contiene valores entre $0$ y $1$.
    - Determina qué tanto del estado oculto anterior es relevante para calcular el nuevo candidato.
    - Valores cercanos a $0$ indican que el estado anterior debe ser "olvidado" al calcular el candidato.
    - Valores cercanos a $1$ indican que el estado anterior es altamente relevante.

    $$\Gamma_r = \sigma \left(W_r \left[h^{\langle t-1\rangle}; x^{\langle t\rangle}\right] + b_r\right)  \tag{2}$$

-  *Estado oculto candidato* $\tilde{h}^{\langle t\rangle}$: es un tensor que contiene la nueva información que podría ser incorporada al estado oculto.
    - Como los datos están normalizados, este tensor contiene valores entre $-1$ y $1$.
    - La compuerta de relevancia $\Gamma_r$ modula cuánto del estado anterior se usa para calcular este candidato.

    $$\tilde{h}^{\langle t\rangle} = \tanh\left(W_h \left[\Gamma_r * h^{\langle t-1\rangle}; x^{\langle t\rangle}\right] + b_h\right) \tag{3} $$

-  *Estado oculto* $h^{\langle t\rangle}$: es un tensor que sirve como entrada para el siguiente paso de tiempo.
    - Es usado para determinar las compuertas del siguiente paso del tiempo.
    - Es una combinación lineal entre el estado oculto anterior y el candidato, ponderada por la compuerta de actualización.
    - Cuando $\Gamma_u \approx 0$, se conserva principalmente $h^{\langle t-1\rangle}$.
    - Cuando $\Gamma_u \approx 1$, se incorpora principalmente $\tilde{h}^{\langle t\rangle}$.

    $$h^{\langle t\rangle} = \Gamma_u * \tilde{h}^{\langle t\rangle} + (1 - \Gamma_u) * h^{\langle t-1\rangle} \tag{4}$$

<a name="4-1"></a>
### Ejercicio 10 - Celda del Forward Pass
Implementa la celda GRU observada en la Figura.

**Instrucciones**:
1. Concatena el estado oculto $h^{\langle t-1 \rangle}$ y $x^{\langle t \rangle}$ en una sola matriz:  

$$concat = \begin{bmatrix} h^{\langle t-1 \rangle} \\ x^{\langle t \rangle} \end{bmatrix}$$  

2. Computa las fórmulas (1) y (2) para las compuertas $\Gamma_u$ y $\Gamma_r$.

3. Concatena el estado oculto modulado $\Gamma_r * h^{\langle t-1 \rangle}$ y $x^{\langle t \rangle}$ para calcular el candidato:

$$concat\_candidate = \begin{bmatrix} \Gamma_r * h^{\langle t-1 \rangle} \\ x^{\langle t \rangle} \end{bmatrix}$$

4. Computa las fórmulas (3) y (4) para el estado oculto candidato y el estado oculto final.

5. Para este problema, no tienes que computar la predicción $y^{\langle t \rangle}$ en todas las celdas, luego no deberás implementarla aquí.

#### Pistas
* Puedes usar [numpy.concatenate](https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html).  Revisa qué valor usar para el parámetro `axis`.
* Las funciones `sigmoid()` y `softmax` se importan desde `utils.py`.
* La documentación sobre [numpy.tanh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tanh.html)
* Usa [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) para la multiplicación de matrices.


In [ ]:
# Función calificadora: gru_cell_forward

def gru_cell_forward(xt, h_prev, parameters):
    """
    Implementa un solo paso hacia adelante de la celda GRU como se describe en la Figura (4)

    Argumentos:
    xt -- tus datos de entrada en el paso de tiempo "t", numpy array de forma (n_x, m).
    h_prev -- Estado oculto en el paso de tiempo "t-1", numpy array de forma (n_h, m)
    parameters -- diccionario de python que contiene:
                        Wu -- Matriz de pesos de la compuerta de actualización, numpy array de forma (n_h, n_h + n_x)
                        bu -- Sesgo de la compuerta de actualización, numpy array de forma (n_h, 1)
                        Wh -- Matriz de pesos del valor candidato, numpy array de forma (n_h, n_h + n_x)
                        bh --  Sesgo del valor candidato, numpy array de forma (n_h, 1)
                        Wr -- Matriz de pesos de la compuerta de reinicio, numpy array de forma (n_h, n_h + n_x)
                        br -- Sesgo de la compuerta de reinicio, numpy array de forma (n_h, 1)


    Returns:
    h_next -- siguiente estado oculto, de forma (n_h, m)
    cache -- tupla de valores necesarios para el paso hacia atrás, contiene (h_next, h_prev, ut, rt, hht, xt, parameters)

    Nota: ut/rt significan las compuertas de actualización/reinicio, hh significa el valor candidato (h tilde).
    """

    # Recuperar parámetros de "parameters"
    Wu = parameters["Wu"]
    bu = parameters["bu"]
    Wh = parameters["Wh"]
    bh = parameters["bh"]
    Wr = parameters["Wr"]
    br = parameters["br"]


    # Recuperar dimensiones de las formas de xt
    n_x, m = xt.shape
    n_h, _ = h_prev.shape # Asumiendo que la forma de h_prev es (n_h, m)

    ### START CODE HERE ###
    # Concatenar h_prev y xt
    concat =

    # Calcular valores para ut, rt, hht, h_next usando las fórmulas dadas en la figura (4)
    ut =
    rt =
    ht =
    concat_candate =
    hht =
    h_next =
    ### END CODE HERE ###

    # almacenar valores necesarios para la propagación hacia atrás en la caché
    cache = (h_next, h_prev, ut, rt, hht, xt, parameters)

    return h_next, cache

In [ ]:
gru_cell_forward_test(gru_cell_forward)

<a name="4-2"></a>
### Ejercicio 11 - Forward Pass
Ya has implementado un paso de una GRU, ahora puedes iterar sobre toda la secuencia utilizando un bucle sobre todas las entradas $T_x$.

Implementa el modelo GRU iterando las celdas hasta los $T_x$ pasos de tiempo.

**Instrucciones**:
-  Obtén las dimensiones $n_x,n_h,n_y,m,T_x$ de las formas de las variables `x` y `parameters`.
-  Inicializa los tensores 3D $h$ y $y$
    -  $h$: estado oculto, forma $(n_h,m,T_x)$.
    -  $y$: predicción, forma $(n_h,m,T_y)$ donde $T_y=1$.
-  Inicializa el tensor 2D $h^{\langle t\rangle}$.
    -  $h^{\langle t\rangle}$ guarda el estado oculto para el paso de tiempo $t$. El nombre de la variable es `h_next`.
    -  $h^{\langle 0\rangle}$, el estado oculto en el tiempo $0$, se pasa cuando se llama la función. El nombre de la variable es `h0`.
    -  Inicializa $h^{\langle t\rangle}$ con el estado oculto $h^{\langle 0\rangle}$ que se pasa a la función.
-  Para cada instante de tiempo, realiza lo siguiente
    -  Desde el tensor 3D $x$, toma la parte 2D $x^{\langle t\rangle}$ en el instante de tiempo $t$.
    -  Llama la función `gru_cell_forward` definida anteriormente, para obtener el estado oculto y el cache.
    -  Guarda el estado oculto dentro del tensor 3D.
    -  Agrega el cache a los caches.

*Nota*: En este caso observa que la predicción se retorna directamente como un tensor 2D.

In [ ]:
# Función calificadora: gru_forward

def gru_forward(x, h0, parameters):
    """
    Implementa la propagación hacia adelante de la red neuronal recurrente usando una celda GRU descrita en la Figura (4).

    Argumentos:
    x -- Datos de entrada para cada paso de tiempo, de forma (n_x, m, T_x).
    h0 -- Estado oculto inicial, de forma (n_h, m)
    parameters -- diccionario de python que contiene:
                        Wu -- Matriz de pesos de la compuerta de actualización, numpy array de forma (n_h, n_h + n_x)
                        bu -- Sesgo de la compuerta de actualización, numpy array de forma (n_h, 1)
                        Wh -- Matriz de pesos del valor candidato, numpy array de forma (n_h, n_h + n_x)
                        bh -- Sesgo del valor candidato, numpy array de forma (n_h, 1)
                        Wr -- Matriz de pesos de la compuerta de reinicio, numpy array de forma (n_h, n_h + n_x)
                        br -- Sesgo de la compuerta de reinicio, numpy array de forma (n_h, 1)
                        Wy -- Matriz de pesos que relaciona el estado oculto con la salida, numpy array de forma (n_y, n_h)
                        by -- Sesgo que relaciona el estado oculto con la salida, numpy array de forma (n_y, 1)

    Returns:
    h -- Estados ocultos para cada paso de tiempo, numpy array de forma (n_h, m, T_x)
    y -- Predicciones para cada paso de tiempo, numpy array de forma (n_y, m)
    caches -- tupla de valores necesarios para el paso hacia atrás, contiene (lista de todas las cachés, x)
    """

    # Inicializar "caches", que rastreará la lista de todas las cachés
    caches = []

    Wy = parameters['Wy']
    by = parameters['by']

    ### START CODE HERE ###
    # Recuperar dimensiones de las formas de x y parameters['Wy']
    n_x, m, T_x =
    n_y, n_h =

    # inicializar "h" y "y" con ceros
    h =
    y =

    # Inicializar h_next
    h_next =

    # iterar sobre todos los pasos de tiempo
    for t in range(T_x): # Iterar a través de todos los pasos de tiempo
        # Obtener la sección 2D 'xt' de la entrada 3D 'x' en el paso de tiempo 't'
        xt =
        # Actualizar el siguiente estado oculto, calcular la predicción, obtener la caché
        h_next, cache =

        # Guardar el valor del nuevo estado oculto "siguiente" en h
        h[:,:,t] =
        # Añadir la caché a caches
        caches.append( ??? )

    ### END CODE HERE ###
    y = np.dot(Wy,h[:,:,-1]) + by
    # almacenar valores necesarios para la propagación hacia atrás en la caché
    caches = (caches, x)

    return h, y, caches

In [ ]:
gru_cell_forward_test(gru_cell_forward)

<a name="4-3"></a>
### Ejercicio 12 - Celda del Backward pass
Ya con la propagación hacia adelante programada, ahora podemos calcular la retropropagación hacia atrás para el entrenamiento del modelo GRU.
<center>
<img src="https://raw.githubusercontent.com/MateoOrtiz001/Laboratory-LSTM-GRU-Inflation/main/imagenes/GRU02.png" width="500">
</center>
En cada celda calculamos la derivada de las variables asociadas a los pesos y sesgos de las compuertas, y las derivadas de la entrada $x^{\langle t\rangle}$ y del estado anterior $h^{\langle t-1\rangle}$.

Como la retropropagación se hace hacia atrás de forma iterativa desde el instante $T_x$, en cada paso tenemos $dh^{\langle t\rangle}$, llamado en el código `dh_next`. Con esto, las derivadas de cada compuerta está dada como sigue.

\begin{align}
d\tilde{h}^{\langle t \rangle} &= dh^{\langle t \rangle} * \Gamma_u^{\langle t \rangle} * \left(1-\left(\tilde{h}^{\langle t \rangle}\right)^2\right)\tag{1} \\[8pt]
du^{\langle t \rangle} &= \left(dh^{\langle t\rangle} * \tilde{h}^{\langle t \rangle} - dh^{\langle t\rangle} * h^{\langle t-1\rangle}\right) * \Gamma_u^{\langle t \rangle} * \left(1-\Gamma_u^{\langle t \rangle}\right)\tag{2} \\[8pt]
dr^{\langle t \rangle} &= \left(W_h^T d\tilde{h}^{\langle t \rangle} * h^{\langle t-1\rangle}\right) * \Gamma_r^{\langle t \rangle} * \left(1-\Gamma_r^{\langle t \rangle}\right)\tag{3}
\end{align}

Con esto podemos calcular las derivadas de los parámetros. Primero, la derivada de los pesos está dada como

$ dW_r = dr^{\langle t \rangle} \begin{bmatrix} h^{\langle t-1\rangle} \\ x^{\langle t\rangle}\end{bmatrix}^T \tag{4} $

$ dW_u = du^{\langle t \rangle} \begin{bmatrix} h^{\langle t-1\rangle} \\ x^{\langle t\rangle}\end{bmatrix}^T \tag{5} $

$ dW_h = d\tilde{h}^{\langle t \rangle} \begin{bmatrix} \Gamma_r^{\langle t\rangle} * h^{\langle t-1\rangle} \\ x^{\langle t\rangle}\end{bmatrix}^T \tag{6}$

Para calcular los sesgos $db_r, db_u, db_h$ solo necesitas sumar a través de todas las 'm' muestras (axis= 1) de $dr^{\langle t \rangle}, du^{\langle t \rangle}, d\tilde{h}^{\langle t \rangle}$ respectivamente. Observa que tienes que tener `keepdims = True`.

$\displaystyle db_r = \sum_{batch}dr^{\langle t \rangle}\tag{7}$
$\displaystyle db_u = \sum_{batch}du^{\langle t \rangle}\tag{8}$
$\displaystyle db_h = \sum_{batch}d\tilde{h}^{\langle t \rangle}\tag{9}$

Finalmente, debes calcular la derivada con respecto al estado anterior y la entrada.

$ dh^{\langle t-1\rangle} = W_r^T dr^{\langle t \rangle} + W_u^T du^{\langle t \rangle} + W_h^T d\tilde{h}^{\langle t \rangle} * \Gamma_r^{\langle t \rangle} + dh^{\langle t\rangle} * (1 - \Gamma_u^{\langle t \rangle}) \tag{10}$

Aquí, para referirnos a la concatenación de los pesos para la ecuación (10) son los primeros $n_h$ valores, es decir, $W_r = W_r[:,:n_h]$, etc.

$ dx^{\langle t \rangle} = W_r^T dr^{\langle t \rangle} + W_u^T du^{\langle t \rangle} + W_h^T d\tilde{h}^{\langle t \rangle}\tag{11} $

en donde los pesos para la ecuación (11) van desde $n_h$ hasta el final, es decir $W_r = W_r[:,n_h:]$, etc.

**Instrucciones:**
-  Implementa las ecuaciones (1) - (11) de arriba.
    - $d\tilde{h}^{\langle t \rangle}$ es representado por `dhht`,    
    - $du^{\langle t \rangle}$ es representado por `dut`,  
    - $dr^{\langle t \rangle}$ es representado por `drt`.

In [ ]:
# Función calificadora: gru_cell_backward

def gru_cell_backward(dh_next, cache):
    """
    Implementa el paso hacia atrás para la celda GRU (un solo paso de tiempo).

    Argumentos:
    dh_next -- Gradientes del siguiente estado oculto, de forma (n_h, m)
    cache -- caché que almacena información del paso hacia adelante

    Returns:
    gradients -- diccionario de python que contiene:
                        dxt -- Gradiente de los datos de entrada en el paso de tiempo t, de forma (n_x, m)
                        dh_prev -- Gradiente con respecto al estado oculto anterior, numpy array de forma (n_h, m)
                        dWr -- Gradiente con respecto a la matriz de pesos de la compuerta de reinicio, numpy array de forma (n_h, n_h + n_x)
                        dbr -- Gradiente con respecto a los sesgos de la compuerta de reinicio, de forma (n_h, 1)
                        dWu -- Gradiente con respecto a la matriz de pesos de la compuerta de actualización, numpy array de forma (n_h, n_h + n_x)
                        dbu -- Gradiente con respecto a los sesgos de la compuerta de actualización, de forma (n_h, 1)
                        dWh -- Gradiente con respecto a la matriz de pesos del estado oculto candidato, numpy array de forma (n_h, n_h + n_x)
                        dbh -- Gradiente con respecto a los sesgos del estado oculto candidato, de forma (n_h, 1)
    """

    # Recuperar información de "cache"
    (h_next, h_prev, ut, rt, hht, xt, parameters) = cache

    Wh = parameters['Wh']
    Wu = parameters['Wu']
    Wr = parameters['Wr']
    concat_c = np.concatenate((rt*h_prev,xt),axis=0).T
    ### START CODE HERE ###
    # Recuperar dimensiones de las formas de xt y h_next
    n_x, m =
    n_h, m =

    # Calcular derivadas relacionadas con las compuertas. Sus valores se pueden encontrar observando cuidadosamente las ecuaciones (7) a (10)
    dhht =
    dut =
    drt =


    # Calcular derivadas relacionadas con los parámetros. Usar ecuaciones (11)-(18)
    concat_t =
    dWr =
    dWu =
    dWh =
    dbr =
    dbu =
    dbh =

    # Calcular derivadas con respecto al estado oculto anterior y entrada. Usar ecuaciones (19)-(21).
    dh_prev =
    dxt =

    ### END CODE HERE ###

    # Guardar gradientes en el diccionario
    gradients = {"dxt": dxt, "dh_prev": dh_prev, "dWr": dWr,"dbr": dbr, "dWu": dWu,"dbu": dbu,
                 "dWh": dWh, "dbh": dbh}

    return gradients

In [ ]:
gru_cell_backward_test(gru_cell_backward,gru_cell_forward)

<a name="4-4"></a>
### Ejercicio 13 - Backward pass
Calcular los gradientes del costo con respecto a $h^{\langle t\rangle}$ en cada paso de tiempo $t$ es útil porque ayuda a retropropagar el gradiente a la celda LSTM anterior. Para hacerlo, tienes que iterar a través de todos los pasos de tiempo empezando por el final $T_x$, y en cada paso, incrementar las derivadas de los parámetros y guardar $dx$.

**Instrucciones**:
-  Crea un bucle empezando desde $T_x$ retrocediendo. Para cada paso, llama tu función `gru_cell_backward` y actualiza los antiguos gradientes sumando los nuevos gradientes.
    -  Observa que `dxt`no se actualiza, pero sí se guarda.

In [ ]:
# Función calificadora: gru_backward

def gru_backward(dh, caches, m_batch):

    """
    Implementa el paso hacia atrás para la RNN con celda GRU (sobre una secuencia completa).

    Argumentos:
    dh -- Gradientes con respecto a los estados ocultos, numpy-array de forma (n_h, m, T_x)
    caches -- caché que almacena información del paso hacia adelante (gru_forward)
    m_batch -- El tamaño del minibatch, entero

    Returns:
    gradients -- diccionario de python que contiene:
                        dx -- Gradiente de las entradas, de forma (n_x, m, T_x)
                        dh0 -- Gradiente con respecto al estado oculto anterior, numpy array de forma (n_h, m)
                        dWr -- Gradiente con respecto a la matriz de pesos de la compuerta de reinicio, numpy array de forma (n_h, n_h + n_x)
                        dWu -- Gradiente con respecto a la matriz de pesos de la compuerta de actualización, numpy array de forma (n_h, n_h + n_x)
                        dWc -- Gradiente con respecto a la matriz de pesos del valor candidato, numpy array de forma (n_h, n_h + n_x)
                        dbr -- Gradiente con respecto a los sesgos de la compuerta de reinicio, de forma (n_h, 1)
                        dbu -- Gradiente con respecto a los sesgos de la compuerta de actualización, de forma (n_h, 1)
                        dbc -- Gradiente con respecto a los sesgos del valor candidato, de forma (n_h, 1)
    """

    # Recuperar valores de la primera caché (t=1) de caches.
    (caches, x) = caches
    (h1, h0, u1, r1, cc1, x1, parameters) = caches[0]

    ### START CODE HERE ###
    # Recuperar dimensiones de las formas de dh y x1
    n_h, m, T_x =
    n_x, m =

    # inicializar los gradientes con los tamaños correctos
    dx =
    dh0 =
    dh_prevt =
    dWu =
    dWr =
    dWh =
    dbu =
    dbr =
    dbh =


    # iterar hacia atrás sobre toda la secuencia
    for t in reversed(range(T_x)):
        # Calcular todos los gradientes usando gru_cell_backward.
        gradients =
        # Almacenar o añadir el gradiente al gradiente del paso anterior de los parámetros
        dh_prevt =
        dx[:,:,t] =
        dWr +=
        dWu +=
        dWh +=
        dbu +=
        dbr +=
        dbh +=
    # Establecer el gradiente de la primera activación al gradiente retropropagado dh_prev.
    dh0 =

    ### END CODE HERE ###
    dWr /= m_batch; dWu /= m_batch; dWh /= m_batch
    dbr /= m_batch; dbu /= m_batch; dbh /= m_batch
    # Almacenar los gradientes en un diccionario de python
    gradients = {"dx": dx, "dh0": dh0, "dWr": dWr,"dbr": dbr, "dWu": dWu,"dbu": dbu,
                 "dWh": dWh, "dbh": dbh}

    return gradients

In [ ]:
gru_backward_test(gru_backward,gru_cell_backward,gru_cell_forward)

### Inicialización de Parámetros de GRU
La inicialización de los parámetros para el modelo GRU es exactamente la misma que utilizamos para inicializar los parámetros en LSTM. Solo que esta vez debes tener en cuenta que los pesos y los sesgos son distintos.

#### Pistas:
-  Recuerda inicializar los pesos utilizando [np.random.randn](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html), que tiene distribución Gaussiana $\mathcal{N}(0,1)$, y a dicho tensor aplicarle el producto con la desviación estándar.

$$ \sigma = \sqrt{\frac{2}{N}}\tag{1} $$

In [ ]:
# Función calificadora: initialize_GRU_parateres

def initialize_GRU_parameters(n_h, n_x, n_y):
    """
    Inicializa los parámetros de la GRU con Xavier initialization

    Arguments:
    n_h -- número de unidades en el hidden state
    n_x -- número de features de entrada
    n_y -- número de features de salida

    Returns:
    parameters -- diccionario con los parámetros inicializados
    """
    np.random.seed(1)

    parameters = {}

    ### START CODE HERE ###

    parameters['Wu'] = np.random.randn(n_h, n_h + n_x) * np.sqrt(2.0 / (n_h + n_x))
    parameters['bu'] = np.zeros((n_h, 1))

    parameters['Wr'] = np.random.randn(n_h, n_h + n_x) * np.sqrt(2.0 / (n_h + n_x))
    parameters['br'] = np.zeros((n_h, 1))

    parameters['Wh'] = np.random.randn(n_h, n_h + n_x) * np.sqrt(2.0 / (n_h + n_x))
    parameters['bh'] = np.zeros((n_h, 1))

    parameters['Wy'] = np.random.randn(n_y, n_h) * np.sqrt(2.0 / n_h)
    parameters['by'] = np.zeros((n_y, 1))

    ### END CODE HERE ###

    return parameters

In [ ]:
initialize_GRU_parameters_test(initialize_GRU_parameters)

<a name="4-5"></a>
### Ejercicio 14 - Entrenamiento del modelo GRU
Ya tienes el modelo para hacer propagación hacia adelante y retropropagación hacia atrás. Ahora, con la inicialización de parámetros, es momento de implementar el entrenamiento del modelo GRU.

En esta implementación vamos a utilizar el algoritmo de adam como optimizador con mini lotes -no te preocupes de esto, pues ya viene implementado y tu solo deberás programar los cálculos de la propagación y la retropropagación- y MSE como función de costo.

Para este ejercicio es importante saber que nuestra función de costo se define

$$\mathcal{L} = \frac{1}{2m_{batch}} \sum_{i\in batch}\left(y_{pred} - y_{real}\right)^2 \tag{1}$$

así, la derivada de esta función con respecto a la predicción $y_{pred}$ es de la forma

$$d\hat y =\frac{d\mathcal L}{d\hat y} = \frac{1}{m_{batch}} \sum_{i\in batch}\left( y_{pred} - y_{real} \right) \tag{2}$$

Con esto podemos calcular los parámetros de la salida y los valores del último paso del estado oculto $h^{\langle T_x\rangle}$. Su derivada es:

$ d h^{\langle T_x\rangle} = W_y^T d \hat y\tag{3}$

Y la derivada de los parámetros de $\hat y$ se calculan de forma similar a los parámetros de las compuertas:

$$ d W_y = d\hat y (h^{\langle t\rangle})^T \tag{4}$$
$$ d b_y = \sum_{batches} d \hat y \tag{5}$$

Como ahora manejamos lotes, el número de muestras está dado por `m_batch`.

**Instrucciones**:
-   Obtén los valores del estado oculto `h`, la predicción `y_pred` y el cache `cache`, calculando la propagación hacia adelante.
    - El estado oculto `h` tiene la forma `(n_h,m_batch,T_x)`.
    - La predicción `y_pred` tiene la forma `(n_y,m_batch)`.
-   Calcula el error de predicción llamando la función `compute_cost()` con las variables `y_pred` y `minibatch_Y`.
    - La variable del mini lote `minibatch_Y` tiene forma `(n_y,m_batch,T_x)`.
-   Inicializa la derivada $d h$ en la variable `dh`.
    - Esta variable tiene la forma `(n_h,m_batch,T_x)`.
-   Calcula la derivada de la función de costo utilizando la ecuación (2) en la variable `dcost_dy_pred`.
-   Calcula la derivada del estado oculto $h^{\langle T_x\rangle}$ utilizando la ecuación (3).
    -   Accede al paso $T_x$ con `h[:,:,-1]`.
    -   Los valores de la propagación se han extraído anteriormente.
-   Obtén los gradientes calculando la retropropagación.
-   Actualiza el valor de las derivadas de los parámetros $W_y$ y $b_y$ utilizando las ecuaciones (4) y (5).

#### Pistas:
-  La función [np.zeros_like](https://numpy.org/devdocs/reference/generated/numpy.zeros_like.html) puede ser útil para inicializar los tensores.
-  Utiliza la función [np.squeeze](https://numpy.org/devdocs/reference/generated/numpy.squeeze.html) para igualar dimensiones entre las variables `y_pred` y `minibatch_Y`.

In [ ]:
# Función calificadora: model_gru_adam

def model_gru_adam(X, Y, n_h, learning_rate=0.001, mini_batch_size=32,
                    beta1=0.9, beta2=0.999, epsilon=1e-8,
                    num_epochs=1000, print_cost=True):
  """
  Entrena una GRU desde cero con Adam y minibatches.

  Arguments:
  X -- datos de entrada, numpy array (n_x, m, T_x)
  Y -- datos objetivo, numpy array (n_y, m, 1)
  n_a -- número de unidades del estado oculto
  learning_rate -- tasa de aprendizaje Adam
  mini_batch_size -- tamaño de minibatch
  num_epochs -- número de épocas
  """

  n_x, m, T_x = X.shape
  n_y, _, _ = Y.shape # Y la forma es (n_y, m, 1)

  # Inicialización
  parameters = initialize_GRU_parameters(n_h, n_x, n_y)
  v, s = initialize_adam(parameters)
  seed = 10
  costs = []
  t = 0  # contador de pasos de Adam

  for epoch in range(num_epochs):
    seed += 1
    minibatches = random_mini_batches(X, Y, mini_batch_size, seed)
    epoch_cost = 0

    for minibatch_X, minibatch_Y in minibatches:
      # Inicialización del estado oculto
      m_batch = minibatch_X.shape[1]
      h0 = np.zeros((n_h, m_batch))

      ### START CODE HERE ###
      # Forward pass utilizando tu función implementada anteriormente gru_forward(minibatch_X, h0, parameters)
      h, y_pred, caches =

      # Costo (MSE) - Calcula y_pred (n_y, m) con minibatch_Y (n_y, m, 1), utiliza .squeeze(axis=2) para igualar dimensiones.
      cost =

      # Inicializa las derivadas del estado oculto
      dh =  # forma (n_h, m_batch, T_x)

      # Calcula la derivada de la predicción de acuerdo a la ecuación (2). Recuerda utilizar .squeeze(axis=2) para igualar dimensiones.
      dcost_dy_pred =

      # Propaga la derivada al último bloque, utiliza la ecuación (3).
      dh[:, :, -1] =

      # Backward pass utilizando tu función implementada anteriormente lstm_backward(dh, caches, m_batch).
      grads =

      # Calcula las derivadas de los parámetros de la salida, utiliza las ecuaciones (4)-(5).
      grads['dWy'] =
      grads['dby'] =

      ### END CODE HERE ###

      # Recortar gradientes
      grads = clip_gradients(grads, 5.0)

      # Update Adam
      t += 1
      parameters, v, s = update_parameters_with_adam(parameters, grads, v, s, t,
                                                        learning_rate, beta1, beta2, epsilon)

      epoch_cost += cost / len(minibatches)

    # Guardar costo
    if print_cost:
      print(f"Cost after epoch {epoch}: {epoch_cost:.6f}")
      costs.append(epoch_cost)

  # Graficar costos
  if print_cost:
    plot_training(costs)


  return parameters

In [ ]:
model_gru_adam_test(model_gru_adam)

Con esto ya puedes entrenar tu modelo GRU. Eres libre de experimentar con distintos hiperparámetros.

In [ ]:
modeloGRU = model_gru_adam(
    X_train,
    y_train,
    n_h= 78,
    learning_rate=0.001,
    mini_batch_size=64,
    num_epochs=200,
    print_cost=True
)


<a name="4-6"></a>
### Predicción del modelo GRU
Listo, ya haz entrenado tu modelo GRU. Sin embargo, ahora queremos hacer predicciones con este. Tu función `gru_forward` realiza una sola predicción sobre el número $T_x$ de pasos. Sin embargo, si queremos realizar predicciones sobre más instantes de tiempo, debemos actualizar las ventanas de tiempo y los valores de las ventanas con los valores predichos.

Esta función es prácticamente la misma que ya haz implementado para LSTM, por lo que no deberás implementarla.

In [ ]:
def predict_gru(window, parameters, steps=1):
  """
  Realiza un número de predicciones con el modelo GRU empezando desde una ventana dada.

  Argumentos:
    window -- Ventana de entrada inicial para la predicción, shape (n_x, 1, T_x)
    parameters -- Diccionario de Python con los parámetros entrenados (Wr, br, Wu, bu, ...)
    steps -- Número de pasos de predicción a realizar

    Retorna:
    future_GRU_predictions -- Una lista con las predicciones.
  """
  # Extraer dimensiones de la entrada.
  n_x, m, T_x = window.shape
  n_y, n_h = parameters["Wy"].shape

  # Lista para guardar las predicciones.
  future_GRU_predictions_scaled = []
  # Inicializa el primer estado oculto con ceros.
  h0 = np.zeros((n_h, 1))

  # Itera hasta el número de pasos dado en el parámetro steps.
  for _ in range(steps):
    # Realiza la prediccion sobre el conjunto de datos y de parámetros.
    h, y, caches = gru_forward(window, h0, parameters)

    # Guarda la prediccion, aplana el tensor 2D utilizando .flatten().
    future_GRU_predictions_scaled.append(y.flatten()[0])

    # Actualiza los valores de la ventana.
    new_window_values = np.append(window.flatten()[1:], y.flatten())

    # Redimensiona la ventana actual para mantener la forma (1,1,T_x).
    window = new_window_values.reshape(1, 1, T_x)

  #Desnormaliza los datos
  future_GRU_predictions = scaler.inverse_transform(np.array(future_GRU_predictions_scaled).reshape(-1, 1))

  return future_GRU_predictions.flatten()

In [ ]:
# Tomamos la última ventana de datos conocida del conjunto de prueba
current_window = X_test[:, -1:, :]
test_GRU_prediction = predict_gru(current_window,modeloGRU,24)


print("Predicciones de inflación para los próximos 24 meses:")
print(test_GRU_prediction.flatten())

In [ ]:
# Des-normalizar los datos para la gráfica
y_train_actual = scaler.inverse_transform(y_train_keras.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test_keras.reshape(-1, 1))

# Crea un índice de fechas para la gráfica
# Los índices de las secuencias corresponden a la *última* fecha de cada ventana.
# Por lo tanto, y_train[i] corresponde a data.index[i + n_steps_in].
start_train_index = n_steps_in
end_train_index = start_train_index + train_size
train_dates = data.index[start_train_index:end_train_index]

start_test_index = end_train_index
end_test_index = start_test_index + len(y_test_keras)
test_dates = data.index[start_test_index:end_test_index]
# Crear fechas futuras para las predicciones
# La primera fecha futura es el mes siguiente a la última fecha de prueba
future_dates = pd.date_range(start=test_dates[-1], periods=25, freq='ME')[1:]
# Graficar los resultados
plt.figure(figsize=(15, 7))

# Datos originales
plt.plot(train_dates, y_train_actual, label='Datos de Entrenamiento (Reales)', color='blue')
plt.plot(test_dates, y_test_actual, label='Datos de Prueba (Reales)', color='green')

# Predicciones
# future_predictions ya está desnormalizado
plt.plot(future_dates, test_GRU_prediction, label='Predicción a 24 Meses', color='red', linestyle='--')

# Estilo del gráfico
plt.title('Predicción de Inflación con GRU', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valor de Inflación', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

<a name="5"></a>
## Implementación con Keras
Ahora vamos a realizar una comparación entre nuestro modelo y uno creado con Keras.

Primero cargamos las librerias necesarias.

#### Implementación de LSTM

In [ ]:
modeloLSTM_Keras = Sequential()
modeloLSTM_Keras.add(LSTM(units=78, input_shape=(X_train.shape[1], 1)))
modeloLSTM_Keras.add(Dense(1))

modeloLSTM_Keras.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
historyLSTM = modeloLSTM_Keras.fit(X_train_keras, y_train_keras, epochs=200, batch_size=64, verbose=1)

In [ ]:
# Graficar la función de pérdida
plt.plot(historyLSTM.history['loss'])
plt.title('Pérdida del modelo durante el entrenamiento')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.show()

In [ ]:
# Predicción para el próximo año
seq_lenght = 12 # n_steps_in

# Tomar la última secuencia de los datos escalados para la predicción inicial
current_window_keras = inflation_scaled[-n_steps_in:].reshape(1, n_steps_in, 1)

# Lista para almacenar las predicciones de 24 meses
future_LSTM_keras_predictions_scaled = []

for _ in range(24):
    # Hacer una predicción para el siguiente paso
    next_step_pred_scaled = modeloLSTM_Keras.predict(current_window_keras, verbose=0)

    # Almacenar la predicción
    future_LSTM_keras_predictions_scaled.append(next_step_pred_scaled[0, 0])

    # Actualizar la ventana: eliminar el valor más antiguo y añadir la nueva predicción
    # Asegurar que current_window_keras permanezca (1, n_steps_in, 1)
    new_window_values = np.append(current_window_keras.flatten()[1:], next_step_pred_scaled.flatten())
    current_window_keras = new_window_values.reshape(1, n_steps_in, 1)

# Desnormalizar todas las 24 predicciones
future_LSTM_keras = scaler.inverse_transform(np.array(future_LSTM_keras_predictions_scaled).reshape(-1, 1))

print("Predicciones de inflación para los próximos 24 meses con Keras:")
print(future_LSTM_keras.flatten())

In [ ]:
# Des-normalizar los datos para la gráfica
y_train_actual = scaler.inverse_transform(y_train_keras.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test_keras.reshape(-1, 1))

# Crea un índice de fechas para la gráfica
# Los índices de las secuencias corresponden a la *última* fecha de cada ventana.
# Por lo tanto, y_train[i] corresponde a data.index[i + n_steps_in].
start_train_index = n_steps_in
end_train_index = start_train_index + train_size
train_dates = data.index[start_train_index:end_train_index]

start_test_index = end_train_index
end_test_index = start_test_index + len(y_test_keras)
test_dates = data.index[start_test_index:end_test_index]
# Crear fechas futuras para las predicciones
# La primera fecha futura es el mes siguiente a la última fecha de prueba
future_dates = pd.date_range(start=test_dates[-1], periods=25, freq='ME')[1:]
# Graficar los resultados
plt.figure(figsize=(15, 7))

# Datos originales
plt.plot(train_dates, y_train_actual, label='Datos de Entrenamiento (Reales)', color='blue')
plt.plot(test_dates, y_test_actual, label='Datos de Prueba (Reales)', color='green')

# Predicciones
# future_predictions ya está desnormalizado
plt.plot(future_dates, future_LSTM_keras.flatten(), label='Predicción a 24 Meses (Keras)', color='red', linestyle='--')

# Estilo del gráfico
plt.title('Predicción de Inflación con LSTM y Keras', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valor de Inflación', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

### Implementación de GRU

In [ ]:
modeloGRU_Keras = Sequential()
modeloGRU_Keras.add(GRU(units=78, input_shape=(X_train.shape[1], 1)))
modeloGRU_Keras.add(Dense(1))

modeloGRU_Keras.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
historyGRU = modeloGRU_Keras.fit(X_train_keras, y_train_keras, epochs=200, batch_size=64, verbose=1)

In [ ]:
# Graficar la función de pérdida
plt.plot(historyGRU.history['loss'])
plt.title('Pérdida del modelo durante el entrenamiento')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.show()

In [ ]:
# Predicción para el próximo año
seq_lenght = 12 # n_steps_in

# Tomar la última secuencia de los datos escalados para la predicción inicial
current_window_keras = inflation_scaled[-n_steps_in:].reshape(1, n_steps_in, 1)

# Lista para almacenar las predicciones de 24 meses
future_GRU_keras_predictions_scaled = []

for _ in range(24):
    # Hacer una predicción para el siguiente paso
    next_step_pred_scaled = modeloGRU_Keras.predict(current_window_keras, verbose=0)

    # Almacenar la predicción
    future_GRU_keras_predictions_scaled.append(next_step_pred_scaled[0, 0])

    # Actualizar la ventana: eliminar el valor más antiguo y añadir la nueva predicción
    # Asegurar que current_window_keras permanezca (1, n_steps_in, 1)
    new_window_values = np.append(current_window_keras.flatten()[1:], next_step_pred_scaled.flatten())
    current_window_keras = new_window_values.reshape(1, n_steps_in, 1)

# Desnormalizar todas las 24 predicciones
future_GRU_keras = scaler.inverse_transform(np.array(future_GRU_keras_predictions_scaled).reshape(-1, 1))

print("Predicciones de inflación para los próximos 24 meses con Keras:")
print(future_GRU_keras.flatten())

In [ ]:
# Des-normalizar los datos para la gráfica
y_train_actual = scaler.inverse_transform(y_train_keras.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test_keras.reshape(-1, 1))

# Crea un índice de fechas para la gráfica
# Los índices de las secuencias corresponden a la *última* fecha de cada ventana.
# Por lo tanto, y_train[i] corresponde a data.index[i + n_steps_in].
start_train_index = n_steps_in
end_train_index = start_train_index + train_size
train_dates = data.index[start_train_index:end_train_index]

start_test_index = end_train_index
end_test_index = start_test_index + len(y_test_keras)
test_dates = data.index[start_test_index:end_test_index]
# Crear fechas futuras para las predicciones
# La primera fecha futura es el mes siguiente a la última fecha de prueba
future_dates = pd.date_range(start=test_dates[-1], periods=25, freq='ME')[1:]
# Graficar los resultados
plt.figure(figsize=(15, 7))

# Datos originales
plt.plot(train_dates, y_train_actual, label='Datos de Entrenamiento (Reales)', color='blue')
plt.plot(test_dates, y_test_actual, label='Datos de Prueba (Reales)', color='green')

# Predicciones
# future_predictions ya está desnormalizado
plt.plot(future_dates, future_GRU_keras.flatten(), label='Predicción a 24 Meses', color='red', linestyle='--')

# Estilo del gráfico
plt.title('Predicción de Inflación con GRU y Keras', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valor de Inflación', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

### Actualizar predicciones (Keras)

In [ ]:
m_test_lstm = X_test.shape[1]
h0_test_lstm = np.zeros((modeloLSTM['Wf'].shape[0], m_test_lstm))

# Realizar predicciones en el conjunto de prueba con el modelo LSTM implementado
_, y_pred_test_lstm_scaled, _, _ = lstm_forward(X_test, h0_test_lstm, modeloLSTM)

# Des-normalizar las predicciones del conjunto de prueba
y_pred_test_lstm_actual = scaler.inverse_transform(y_pred_test_lstm_scaled.reshape(-1, 1))

# Concatenar las predicciones del conjunto de prueba con las futuras
combined_LSTM_predictions = np.concatenate((y_pred_test_lstm_actual.squeeze(), test_LSTM_prediction))


In [ ]:
# Realizar predicciones en el conjunto de prueba con el modelo LSTM de Keras
y_pred_test_lstm_keras_scaled = modeloLSTM_Keras.predict(X_test_keras, verbose=0)

# Des-normalizar las predicciones del conjunto de prueba
y_pred_test_lstm_keras_actual = scaler.inverse_transform(y_pred_test_lstm_keras_scaled.reshape(-1, 1))

# Concatenar las predicciones del conjunto de prueba con las futuras
combined_LSTM_keras_predictions = np.concatenate((y_pred_test_lstm_keras_actual, future_LSTM_keras))


In [ ]:
m_test_gru = X_test.shape[1]
h0_test_gru = np.zeros((modeloGRU['Wu'].shape[0], m_test_gru))

# Realizar predicciones en el conjunto de prueba con el modelo GRU implementado
_, y_pred_test_gru_scaled, _ = gru_forward(X_test, h0_test_gru, modeloGRU)

# Des-normalizar las predicciones del conjunto de prueba
y_pred_test_gru_actual = scaler.inverse_transform(y_pred_test_gru_scaled.reshape(-1, 1))

# Concatenar las predicciones del conjunto de prueba con las futuras
combined_GRU_predictions = np.concatenate((y_pred_test_gru_actual.squeeze(), test_GRU_prediction))


In [ ]:
# Realizar predicciones en el conjunto de prueba con el modelo GRU de Keras
y_pred_test_gru_keras_scaled = modeloGRU_Keras.predict(X_test_keras, verbose=0)

# Des-normalizar las predicciones del conjunto de prueba
y_pred_test_gru_keras_actual = scaler.inverse_transform(y_pred_test_gru_keras_scaled.reshape(-1, 1))

# Concatenar las predicciones del conjunto de prueba con las futuras
combined_GRU_keras_predictions = np.concatenate((y_pred_test_gru_keras_actual, future_GRU_keras))


In [ ]:
y_train_actual = scaler.inverse_transform(y_train_keras.reshape(-1, 1))
start_train_index = n_steps_in
end_train_index = start_train_index + train_size
train_dates = data.index[start_train_index:end_train_index]
start_test_index = end_train_index
end_test_index = start_test_index + len(y_test_keras)
test_dates = data.index[start_test_index:end_test_index]
combined_prediction_dates = pd.date_range(start=test_dates[0], periods=len(y_test_keras) + 24, freq='ME')
plt.figure(figsize=(13, 7))
plt.plot(test_dates, y_test_actual, label='Datos de Prueba (Reales)', color='green',linewidth=2, alpha=0.7)
plt.plot(combined_prediction_dates, combined_LSTM_predictions.flatten(), label='Predicción LSTM (Implementada)', color='red', linestyle='--')
plt.plot(combined_prediction_dates, combined_LSTM_keras_predictions.flatten(), label='Predicción LSTM (Keras)', color='orange', linestyle='--')
plt.plot(combined_prediction_dates, combined_GRU_predictions.flatten(), label='Predicción GRU (Implementada)', color='blue', linestyle='--')
plt.plot(combined_prediction_dates, combined_GRU_keras_predictions.flatten(), label='Predicción GRU (Keras)', color='brown', linestyle='--')
plt.axvline(x=test_dates[-1], color='black', linestyle=':', linewidth=1, label='Fin datos de prueba')
plt.axvline(x=combined_prediction_dates[len(y_test_keras) + 11], color='black', linestyle='--', linewidth=1, label='Fin 12 meses futuros')
plt.xlim(test_dates[0], combined_prediction_dates[-1])
plt.title('Predicciones de Inflación a 24 Meses (Todos los Modelos)', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valor de Inflación', fontsize=12)
plt.legend()
plt.grid(True)

#plt.savefig("predicciones3.png")
plt.show()


<a name="6"></a>
## Predicción en intervalos de confianza (Opcional)

Una vez entrenados los modelos LSTM y GRU, es útil no solo obtener predicciones puntuales sino también cuantificar la incertidumbre asociada a dichas predicciones. Los intervalos de confianza nos permiten establecer un rango dentro del cual esperamos que se encuentre el valor real con cierta probabilidad (típicamente 95%).

Para construir estos intervalos utilizaremos un método de **bootstrap residual**, que consiste en:

1. Calcular los residuos del conjunto de prueba (diferencia entre valores reales y predichos).
2. Generar múltiples trayectorias de predicción futura, donde cada trayectoria se ajusta añadiendo residuos seleccionados aleatoriamente.
3. Calcular la media y los percentiles de todas las trayectorias generadas para obtener la predicción promedio y los límites del intervalo.

Este enfoque captura la variabilidad observada en los errores del modelo durante el entrenamiento y la proyecta hacia las predicciones futuras.

### Calcular Residuos del Conjunto de Prueba

El primer paso es calcular los residuos, que representan los errores que el modelo cometió en el conjunto de prueba. Estos residuos serán la base para estimar la incertidumbre en predicciones futuras.


In [ ]:
residuals_lstm = y_test_actual - y_pred_test_lstm_actual
residuals_gru = y_test_actual - y_pred_test_gru_actual

print("Residuals for implemented LSTM model:\n", residuals_lstm[:5])
print("\nResiduals for implemented GRU model:\n", residuals_gru[:5])

Los residuos muestran qué tan lejos estuvieron las predicciones del modelo de los valores reales. Un buen modelo tendrá residuos pequeños y distribuidos aleatoriamente alrededor de cero.

### Ejercicio Opcional - Función de Bootstrap para Intervalos de Confianza

Implementa la función `bootstrap_prediction_intervals` que genera predicciones futuras con intervalos de confianza utilizando el método de bootstrap residual.

El método funciona de la siguiente manera:
1. Para cada muestra de bootstrap, comenzamos con la última ventana de datos observados.
2. Realizamos predicciones paso a paso hacia el futuro.
3. En cada paso, ajustamos la predicción base del modelo añadiendo un residuo seleccionado aleatoriamente del conjunto de residuos de prueba.
4. Actualizamos la ventana deslizante con la predicción ajustada para continuar con el siguiente paso.
5. Repetimos este proceso múltiples veces (típicamente 100 o más) para generar un conjunto de trayectorias posibles.
6. Finalmente, calculamos la media y los percentiles 2.5% y 97.5% de todas las trayectorias para obtener la predicción promedio y un intervalo de confianza del 95%.

**Instrucciones**:
-  Implementa el bucle interno que realiza predicciones para cada paso futuro (`n_steps_out`).
-  Para cada paso de predicción:
    1. Realiza la predicción base utilizando la función `predict_lstm` o `predict_gru` según el tipo de modelo.
        - Usa `current_window_bootstrap` como entrada.
        - La predicción estará escalada (normalizada).
    2. Selecciona un residuo aleatorio del vector `residuals_flat` utilizando `np.random.choice()`.
        - Este residuo también está escalado, por lo que es compatible con la predicción escalada.
    3. Ajusta la predicción base sumándole el residuo aleatorio.
        - Usa `.flatten()[0]` para extraer el valor escalar de la predicción.
        - Guarda el resultado en `adjusted_pred_scaled`.
    4. Agrega la predicción ajustada a la lista `current_bootstrap_predictions_scaled`.
    5. Actualiza la ventana deslizante para la siguiente iteración:
        - Elimina el primer valor de la ventana actual usando `current_window_bootstrap.flatten()[1:]`.
        - Añade la predicción ajustada al final usando `np.append()`.
        - Reorganiza el resultado en forma `(1, 1, n_steps_in)` usando `.reshape()`.

#### Pistas:
-  Usa `np.random.choice(residuals_flat)` para seleccionar un residuo aleatorio con reemplazo.
-  Usa `np.append(array1, value)` para añadir un valor al final de un array.
-  Recuerda que las predicciones de los modelos tienen forma `(n_y, m)`, por lo que necesitas usar `.flatten()[0]` para extraer el valor escalar.
-  La ventana debe mantener siempre el mismo tamaño `n_steps_in`, por lo que debes eliminar el primer elemento antes de añadir el nuevo al final.



In [ ]:
import copy

def bootstrap_prediction_intervals(model, parameters, scaler, last_window, n_steps_in, n_steps_out, residuals, n_bootstrap_samples=100, model_type='lstm'):
    """
    Genera predicciones futuras con intervalos de confianza utilizando un método de bootstrap.

    Argumentos:
    model -- La función de predicción del modelo (e.g., predict_lstm o predict_gru).
    parameters -- Diccionario con los parámetros entrenados del modelo.
    scaler -- El objeto StandardScaler utilizado para normalizar los datos.
    last_window -- La última ventana de datos escalados para iniciar la predicción, shape (1, 1, n_steps_in).
    n_steps_in -- Número de pasos de entrada de la ventana (para el reshape).
    n_steps_out -- Número de pasos a predecir (horizonte de predicción).
    residuals -- Residuos del conjunto de prueba (diferencia entre real y predicho), shape (num_residuals, 1).
    n_bootstrap_samples -- Número de muestras de bootstrap a generar.
    model_type -- Tipo de modelo ('lstm' o 'gru') para seleccionar la función de predicción.

    Retorna:
    mean_predictions -- La predicción media desnormalizada para cada paso futuro, shape (n_steps_out, 1).
    lower_bound -- El límite inferior del intervalo de confianza desnormalizado, shape (n_steps_out, 1).
    upper_bound -- El límite superior del intervalo de confianza desnormalizado, shape (n_steps_out, 1).
    """

    all_bootstrap_predictions_scaled = []
    residuals_flat = residuals.flatten()

    for _ in range(n_bootstrap_samples):
        current_window_bootstrap = copy.deepcopy(last_window)
        current_bootstrap_predictions_scaled = []

        ### START CODE HERE ###
        for _ in range( ??? ):
            # Hacer la predicción base (escalada). Utiliza tus funciones predict_lstm y predict_gru según sea el caso.
            if model_type == 'lstm':
                next_step_pred_scaled =
            else:     # GRU
                next_step_pred_scaled =

            # Seleccionar un residuo aleatorio con reemplazo
            random_residual =

            # Ajustar la predicción con el residuo
            adjusted_pred_scaled =

            current_bootstrap_predictions_scaled.append( ??? )

            # Actualizar la ventana para la siguiente predicción
            # Asegurarse de que el new_window_values tenga la forma correcta antes de reshape
            new_window_values = np.append( ??? )
            current_window_bootstrap =

            ### END CODE HERE ###

        all_bootstrap_predictions_scaled.append(current_bootstrap_predictions_scaled)

    # Convertir la lista de predicciones bootstrap a un array numpy
    all_bootstrap_predictions_scaled = np.array(all_bootstrap_predictions_scaled)

    # Calcular la media y los percentiles para cada paso de tiempo
    mean_predictions_scaled = np.mean(all_bootstrap_predictions_scaled, axis=0).reshape(-1, 1)
    lower_bound_scaled = np.percentile(all_bootstrap_predictions_scaled, 2.5, axis=0).reshape(-1, 1)
    upper_bound_scaled = np.percentile(all_bootstrap_predictions_scaled, 97.5, axis=0).reshape(-1, 1)

    # Desnormalizar las predicciones
    mean_predictions = scaler.inverse_transform(mean_predictions_scaled)
    lower_bound = scaler.inverse_transform(lower_bound_scaled)
    upper_bound = scaler.inverse_transform(upper_bound_scaled)

    return mean_predictions, lower_bound, upper_bound

print("bootstrap_prediction_intervals function defined.")

### Generar Intervalos de Predicción LSTM



In [ ]:
last_window_lstm = X_test[:, -1:, :]

lstm_mean_predictions, lstm_lower_bound, lstm_upper_bound = bootstrap_prediction_intervals(
    model=predict_lstm,
    parameters=modeloLSTM,
    scaler=scaler,
    last_window=last_window_lstm,
    n_steps_in=n_steps_in,
    n_steps_out=24,
    residuals=residuals_lstm,
    n_bootstrap_samples=500,
    model_type='lstm'
)

print("LSTM Mean Predictions (next 24 months):")
print(lstm_mean_predictions.flatten()[:5], "...")
print("\nLSTM Lower Bound (next 24 months):")
print(lstm_lower_bound.flatten()[:5], "...")
print("\nLSTM Upper Bound (next 24 months):")
print(lstm_upper_bound.flatten()[:5], "...")

### Generar Intervalos de Predicción GRU


In [ ]:
last_window_gru = X_test[:, -1:, :]

gru_mean_predictions, gru_lower_bound, gru_upper_bound = bootstrap_prediction_intervals(
    model=predict_gru,
    parameters=modeloGRU,
    scaler=scaler,
    last_window=last_window_gru,
    n_steps_in=n_steps_in,
    n_steps_out=24,
    residuals=residuals_gru,
    n_bootstrap_samples=500,
    model_type='gru'
)

print("GRU Mean Predictions (next 24 months):")
print(gru_mean_predictions.flatten()[:5], "...")
print("\nGRU Lower Bound (next 24 months):")
print(gru_lower_bound.flatten()[:5], "...")
print("\nGRU Upper Bound (next 24 months):")
print(gru_upper_bound.flatten()[:5], "...")

### Graficar Predicciones LSTM con Intervalos


In [ ]:
import matplotlib.pyplot as plt

# Des-normalizar los datos para la gráfica
y_train_actual = scaler.inverse_transform(y_train_keras.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test_keras.reshape(-1, 1))
start_train_index = n_steps_in
end_train_index = start_train_index + train_size
train_dates = data.index[start_train_index:end_train_index]

start_test_index = end_train_index
end_test_index = start_test_index + len(y_test_keras)
test_dates = data.index[start_test_index:end_test_index]
combined_prediction_dates = pd.date_range(start=test_dates[0], periods=len(y_test_keras) + 24, freq='ME')
combined_lstm_bootstrap_mean_predictions = np.concatenate((y_pred_test_lstm_actual, lstm_mean_predictions))
plt.figure(figsize=(7, 6))
plt.plot(test_dates, y_test_actual, label='Datos de Prueba (Reales)', color='green', linewidth=2)
plt.plot(combined_prediction_dates, combined_lstm_bootstrap_mean_predictions.flatten(), label='Predicción LSTM (Implementada) con IC', color='red', linestyle='-')
future_dates_only = pd.date_range(start=test_dates[-1], periods=25, freq='ME')[1:]
plt.fill_between(future_dates_only, lstm_lower_bound.flatten(), lstm_upper_bound.flatten(), color='red', alpha=0.2, label='Intervalo de Confianza LSTM (95%)')
plt.plot(combined_prediction_dates, combined_LSTM_keras_predictions.flatten(), label='Predicción LSTM (Keras)', color='orange', linestyle='--')
plt.axvline(x=test_dates[-1], color='black', linestyle=':', linewidth=1, label='Fin datos de prueba')
plt.axvline(x=combined_prediction_dates[len(y_test_keras) + 11], color='black', linestyle='--', linewidth=1, label='Fin 12 meses futuros')
plt.xlim(test_dates[0], combined_prediction_dates[-1])
plt.title('Predicción de Inflación con LSTM con IC', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valor de Inflación', fontsize=12)
plt.legend()
plt.grid(True)
#plt.savefig("lstmIC.png")
plt.show()

### Graficar Predicciones GRU con Intervalos


In [ ]:
# Des-normalizar los datos para la gráfica
y_train_actual = scaler.inverse_transform(y_train_keras.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test_keras.reshape(-1, 1))

# Crea un índice de fechas para la gráfica
start_train_index = n_steps_in
end_train_index = start_train_index + train_size
train_dates = data.index[start_train_index:end_train_index]

start_test_index = end_train_index
end_test_index = start_test_index + len(y_test_keras)
test_dates = data.index[start_test_index:end_test_index]

# Crear fechas futuras para las predicciones
future_dates_only = pd.date_range(start=test_dates[-1], periods=25, freq='ME')[1:]
combined_prediction_dates = pd.date_range(start=test_dates[0], periods=len(y_test_keras) + 24, freq='ME')
combined_gru_bootstrap_mean_predictions = np.concatenate((y_pred_test_gru_actual, gru_mean_predictions))
plt.figure(figsize=(7, 6))
plt.plot(test_dates, y_test_actual, label='Datos de Prueba (Reales)', color='green', linewidth=2)
plt.plot(combined_prediction_dates, combined_gru_bootstrap_mean_predictions.flatten(), label='Predicción GRU (Implementada) con IC', color='blue', linestyle='-')
plt.fill_between(future_dates_only, gru_lower_bound.flatten(), gru_upper_bound.flatten(), color='blue', alpha=0.2, label='Intervalo de Confianza GRU (95%)')
plt.plot(combined_prediction_dates, combined_GRU_keras_predictions.flatten(), label='Predicción GRU (Keras)', color='c', linestyle='--')
plt.axvline(x=test_dates[-1], color='black', linestyle=':', linewidth=1, label='Fin datos de prueba')
plt.axvline(x=combined_prediction_dates[len(y_test_keras) + 11], color='black', linestyle='--', linewidth=1, label='Fin 12 meses futuros')
plt.xlim(test_dates[0], combined_prediction_dates[-1])
plt.title('Predicción de Inflación con GRU con IC', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Valor de Inflación', fontsize=12)
plt.legend()
plt.grid(True)
#plt.savefig("gruIC.png")
plt.show()

<a name="7"></a>
## Bibliografia

- Ortiz Higuera, M. S. (2025). Construcción de un laboratorio interactivo para el modelamiento de Redes Neuronales Recurrentes con aplicaciones en actuaría y finanzas. *Tesis de pregrado, Universidad Nacional de Colombia*. Facultad de Ciencias, Departamento de Matemáticas, Sede Bogotá.

- Cárdenas-Cárdenas, J. A., Cristiano-Botia, D. J., Martínez-Cortés, N., & others. (2023). Colombian inflation forecast using long short-term memory approach. *Borradores de Economía; No. 1241*. Banco de la República de Colombia.

- Hochreiter, S., & Schmidhuber, J. (1997). Long Short-Term Memory. *Neural Computation, 9*(8), 1735-1780. https://doi.org/10.1162/neco.1997.9.8.1735

- Ng, A., & deeplearning.ai. (2018). Sequence Models. Coursera. https://www.coursera.org/learn/nlp-sequence-models (Accedido el 10 de Septiembre de 2025)

- Cho, K., van Merrienboer, B., Bahdanau, D., & Bengio, Y. (2014). On the Properties of Neural Machine Translation: Encoder-Decoder Approaches. *arXiv preprint arXiv:1409.1259*. https://arxiv.org/abs/1409.1259